# **Identitas Mahasiswa**

---


*   **Nama: Ripan**
*   **NIM: 2310325**
*   **Mata Kuliah: Machine Learning**
*   **Dataset yang Digunakan: TB (Tuberkolosis)**

---






# **Dataset yang Digunakan dalam Pemodelan**
* [🔗 Klik disini untuk melihat Dataset tb_2021_kepesertaan.csv](https://drive.google.com/file/d/1nbd6zlghqanj38iEs_Rs9MBeB2hp298N/view?usp=drive_link).
* [🔗 Klik disini untuk melihat Dataset tb_fktp_nonkapitasi.csv](https://drive.google.com/file/d/1VjkQyyDTlp3q0EYsqG4fHPlN-v6yGP9I/view?usp=drive_link).

---

# **Prediksi Biaya Tagihan Pasien FKTP (Fasilitas Kesehatan Tingkat Pertama) Non-Kapitasi**

FKTP Non-Kapitasi adalah layanan di fasilitas kesehatan tingkat pertama, seperti puskesmas atau klinik yang pembayarannya tidak bersifat tetap per peserta, melainkan berdasarkan tindakan medis yang benar-benar dilakukan. Dalam skema ini, BPJS Kesehatan sebagai badan penyelenggara yang menanggung biaya layanan tersebut, bukan pasien secara langsung.

Prediksi biaya tagihan pasien di FKTP Non-Kapitasi bukan sekadar urusan angka, melainkan upaya memahami bagaimana sistem kesehatan bisa berjalan lebih adil dan efisien. Dengan mengenali faktor-faktor yang membuat biaya meningkat, kita dapat mencegah pemborosan, mendeteksi penagihan yang tidak wajar, serta memastikan dana publik digunakan untuk hal yang benar-benar menyembuhkan. Di tengah meningkatnya tekanan biaya medis, kemampuan memprediksi tagihan menjadi kunci untuk menjaga keseimbangan antara keberlanjutan sistem kesehatan dan kemanusiaan dalam setiap keputusan finansial.


---



# **Langkah-Langkah Membangun Model Prediksi Biaya Tagihan Pasien FKTP Non-Kapitasi dengan Algoritma LGBM (Light Gradient Boosting Machine)**

1.   **Exploratory Data Analysis (EDA)**
* Menentukan Primary Table, Related Table, dan Target Analisis
* Inspeksi Awal dan Pemuatan Data
* Penyamaan Struktur Primary Table dan Related Table Berdasarkan **"PSTV01" (no_peserta)**
* Normalisasi Penamaan dan Penyelarasan Atribut Kolom
* Penggabungan Tabel, Penyeragaman Kolom Duplikat, dan Penataan Struktur Data
2.   **Data Preprocessing**
* Pemeriksaan Missing Value
* Penanganan Missing Value
* Pembuatan Fitur Baru: usia_kunjungan dan lama_rawat
* Identifikasi Outlier
* Penanganan Outlier — ***Catatan: Penanganan outlier bersifat opsional dan tergantung karakteristik data. Misalnya, jika outlier dihapus, dicapping, atau dimodifikasi, variasi nilai dan pengaruhnya terhadap target dapat hilang atau berkurang.***
* Pemeriksaan Distribusi Data
* Analisis Korelasi Numerik dan Kategorikal terhadap Target **"PNK17" (biaya_tagihan)**
3. **Train/Test Split Data**
* Train/Test Split dengan Perbandingan 80:20
4. **Data Modelling**
* Perbandingan Algoritma yang Dapat Menangani Data Kategorikal, Outlier, dan Distribusi Tidak Normal secara Native
* Perbandingan Algoritma yang Memerlukan Pipeline untuk Menangani Data Kategorikal, Outlier, dan Distribusi Tidak Normal
* Pemilihan Algoritma Final dan Pelatihan Ulang Model
* Pemilihan Model Terbaik, Pelatihan Model Final, dan Penyimpanan Model
* Melakukan Prediksi dengan Data Baru
---



## **1. Exploratory Data Analysis (EDA)**
Exploratory Data Analysis (EDA) adalah tahap awal dalam analisis data yang bertujuan untuk memahami struktur, kualitas, dan hubungan antar variabel sebelum melakukan pemodelan. Pada tahap ini, dilakukan penentuan Primary Table, Related Table, dan target analisis, inspeksi awal dan pemuatan data, penyamaan struktur Primary Table dan Related Table berdasarkan **"PSTV01" (no_peserta)**, normalisasi penamaan kolom untuk menghindari duplikasi atau kebingungan, serta penggabungan Primary Table dan Related Table dengan penataan kolom sesuai kebutuhan agar data siap untuk tahap analisis lebih lanjut.

---

### **1.1 Menentukan Primary Table, Related Table, dan Target Analisis**
Target variabel yang digunakan adalah **"PNK17" (biaya_tagihan)**, yang terdapat pada tabel **"tb_fktp_nonkapitasi.csv"**. Tabel ini juga berisi calon-calon prediktor utama. Selain itu, terdapat tabel historis **"tb_2021_kepesertaan.csv"** yang memuat data calon prediktor terkait histori peserta, sehingga tabel ini digunakan sebagai related table untuk memperkaya informasi prediktor.

Tabel-tabel lain, seperti **"tb_fkrtl.csv"** maupun tabel-tabel setelahnya **("tb_fkrtl_diagnosis_sekunder.csv")**, walaupun memiliki beberapa kolom yang berkorelasi, tidak memuat informasi histori yang relevan terhadap target **"PNK17" (biaya_tagihan)**, sehingga dieksklusi dari analisis. Dengan demikian, struktur data yang dipakai hanya terdiri dari:

*   **Primary Table: tb_fktp_nonkapitasi.csv**
*   **Related Table: tb_2021_kepesertaan.csv**

---

### **1.2 Inspeksi Awal dan Pemuatan Data**
Pada tahap ini dilakukan penyiapan lingkungan kerja yang diperlukan, termasuk memuat file data, memeriksa lokasi file, jumlah baris dan kolom, tipe data tiap kolom, serta menampilkan cuplikan data (preview) untuk memahami struktur awal tabel. Selain itu, dilakukan pembersihan awal dengan menghapus kolom Unnamed: 0 yang tidak relevan untuk analisis lebih lanjut.

---

In [ ]:
# -----------------------------------------------------------------------------
# Setup Environment
# -----------------------------------------------------------------------------
!pip install catboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pickle
import joblib
import warnings
from google.colab import drive
from IPython.display import display
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pointbiserialr, f_oneway
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgbm
from scipy.stats import kstest
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_val_score
# -----------------------------------------------------------------------------
# Setup Environment
# -----------------------------------------------------------------------------

# Mengatur seed untuk keterulangan (reproducibility)
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
warnings.filterwarnings('ignore')

print("✅ Library berhasil di-load.")

# -----------------------------------------------------------------------------
# Inspeksi dan Pemuatan Data
# -----------------------------------------------------------------------------

# Daftar file csv yang di ambil dari drive
drive.mount('/content/drive')
file_paths = {
    "Kepesertaan": "/content/drive/MyDrive/Colab Notebooks/SESUAIKAN_DENGAN_PATH_DRIVE_ANDA.csv",
    "Non Kapitasi": "/content/drive/MyDrive/Colab Notebooks/SESUAIKAN_DENGAN_PATH_DRIVE_ANDA.csv",
}

# Opsi tampilan agar tabel terlihat lengkap
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Semua data berhasil dimuat.")

print("\n")
print("—" * 500)
print("\n")

# Proses pemuatan dan inspeksi
clean_file_paths = {}
for name, path in file_paths.items():
    try:
        df = pd.read_csv(path)

        # Hapus kolom Unnamed jika ada
        if 'Unnamed: 0' in df.columns:
            df.drop('Unnamed: 0', axis=1, inplace=True)
            print(f"🗑️ Kolom 'Unnamed: 0' dihapus dari {name}")

        # Simpan CSV baru yang sudah bersih
        clean_path = path.replace(".csv", "_clean.csv")
        df.to_csv(clean_path, index=False)
        clean_file_paths[name] = clean_path

        print(f"\n=== Data: {name} ===")
        print(f"Lokasi file bersih: {clean_path}")
        print(f"Jumlah baris: {df.shape[0]}")
        print(f"Jumlah kolom: {df.shape[1]}\n")
        print("Tipe data setiap kolom:")
        print(df.dtypes)
        print("\nCuplikan data:")
        display(df.head(10))

        print("\n")
        print("—" * 500)
        print("\n")

    except Exception as e:
        print(f"❌ Gagal memuat {name}:", e)

print("✅ Semua data berhasil dibersihkan dan disimpan tanpa kolom Unnamed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.5 MB/s eta 0:00:00
✅ Library berhasil di-load.
Mounted at /content/drive
✅ Semua data berhasil dimuat.


————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————


🗑️ Kolom 'Unnamed: 0' dihapus dari Kepesertaan

=== Data: Kepesertaan ===
Lokasi file bersih: /content/drive/MyDrive/Colab Notebooks/SESUAIKAN_DENGAN_PATH_DRIVE_ANDA.csv
Jumlah baris: 94966
Jumlah kolom: 18

Tipe data setiap kolom:
PSTV01      int64
PSTV02      int64
PSTV03     object
PSTV04     object
PSTV05     object
PSTV06     

,PSTV01,PSTV02,PSTV03,PSTV04,PSTV05,PSTV06,PSTV07,PSTV08,PSTV09,PSTV10,PSTV11,PSTV12,PSTV13,PSTV14,PSTV15,PSTV16,PSTV17,PSTV18
0,32853965,31945523,1958-11-27,SUAMI,LAKI-LAKI,KAWIN,KELAS I,PPU,ACEH,ACEH TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,ACEH TENGAH,1.531137,2021,MENINGGAL,2020.0
1,8555967,8555967,1936-07-01,PESERTA,LAKI-LAKI,KAWIN,KELAS I,BUKAN PEKERJA,ACEH,ACEH TENGAH,POLRI,KLINIK PRATAMA,ACEH,BIREUEN,0.998777,2021,AKTIF,NaN
2,8797049,249479,1950-07-21,SUAMI,LAKI-LAKI,KAWIN,KELAS I,BUKAN PEKERJA,ACEH,ACEH SELATAN,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,ACEH SELATAN,30.096481,2021,TIDAK AKTIF,NaN
3,83472658,54408870,2012-08-12,ANAK,PEREMPUAN,BELUM KAWIN,KELAS I,PPU,ACEH,ACEH BARAT,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,ACEH BARAT,9.837102,2021,AKTIF,NaN
4,425874326,296852327,2004-12-30,ANAK,LAKI-LAKI,BELUM KAWIN,KELAS II,PPU,SUMATERA UTARA,TOBA SAMOSIR,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,SUMATERA UTARA,TOBA SAMOSIR,1.038886,2021,AKTIF,NaN
5,379070359,379070359,1955-03-08,PESERTA,LAKI-LAKI,KAWIN,KELAS I,BUKAN PEKERJA,SUMATERA UTARA,KOTA PEMATANG SIANTAR,SWASTA,KLINIK PRATAMA,SUMATERA UTARA,KOTA PEMATANG SIANTAR,1.528337,2021,MENINGGAL,2019.0
6,71191788,71191788,1963-05-01,PESERTA,PEREMPUAN,KAWIN,KELAS I,PPU,SUMATERA UTARA,SAMOSIR,SWASTA,KLINIK PRATAMA,SUMATERA UTARA,SAMOSIR,2.809494,2021,AKTIF,NaN
7,14278660,14278660,1963-04-08,PESERTA,PEREMPUAN,KAWIN,KELAS I,PPU,SUMATERA UTARA,NIAS SELATAN,SWASTA,KLINIK PRATAMA,SUMATERA UTARA,NIAS SELATAN,1.157796,2021,AKTIF,NaN
8,25390893,25390893,1974-11-11,PESERTA,LAKI-LAKI,KAWIN,KELAS I,PPU,SUMATERA UTARA,MANDAILING NATAL,SWASTA,DOKTER UMUM,SUMATERA UTARA,MANDAILING NATAL,6.672624,2021,AKTIF,NaN
9,280051063,280051063,1965-10-23,PESERTA,PEREMPUAN,KAWIN,KELAS I,PPU,SUMATERA UTARA,TAPANULI SELATAN,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,SUMATERA UTARA,TAPANULI SELATAN,5.830530,2021,MENINGGAL,2019.0




————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————


🗑️ Kolom 'Unnamed: 0' dihapus dari Non Kapitasi

=== Data: Non Kapitasi ===
Lokasi file bersih: /content/drive/MyDrive/Colab Notebooks/SESUAIKAN_DENGAN_PATH_DRIVE_ANDA.csv
Jumlah baris: 36493
Jumlah kolom: 21

Tipe data setiap kolom:
PSTV01      int64
PSTV02      int64
PSTV15    float64
PNK02      object
PNK03      object
PNK04      object
PNK05      object
PNK06      object
PNK07      object
PNK08      object
PNK09      object
PNK10      object
PNK11      object
PNK12      object
PNK13    

,PSTV01,PSTV02,PSTV15,PNK02,PNK03,PNK04,PNK05,PNK06,PNK07,PNK08,PNK09,PNK10,PNK11,PNK12,PNK13,PNK13A,PNK14,PNK15,PNK16,PNK17,PNK18
0,93858078,93216423,7.659537,183920215Y000376,2015-02-26,2015-02-26,2015-02-27,SULAWESI SELATAN,BARRU,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RITP,PBI APBN,NaN,NaN,9999,9999,Evakuasi medis / Ambulans Darat,120000,120000
1,93747649,346217457,1.800196,19500915Y000074,2015-09-21,2015-09-23,2015-09-23,KALIMANTAN SELATAN,TAPIN,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RITP,PBI APBN,NaN,NaN,9999,9999,Rawat Inap di R. Perawatan Biasa,240000,240000
2,359887820,72989971,0.959155,250630919P000299,2019-09-26,2019-09-29,2019-09-29,PAPUA,JAYAPURA,TNI AD,KLINIK PRATAMA,KLINIK RAWAT INAP,RITP,PPU,A01 Typhoid and paratyphoid fevers,A01,A010,Typhoid fever,Rawat Inap di R. Perawatan Biasa,600000,600000
3,84126594,84126594,1.110887,326360919P001086,2019-09-09,2019-09-12,2019-09-12,JAWA TENGAH,REMBANG,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RITP,PBI APBN,A01 Typhoid and paratyphoid fevers,A01,A01,Typhoid and paratyphoid fevers,Rawat Inap di R. Perawatan Biasa,480000,480000
4,87558937,62126532,32.451830,252721019P001142,2019-10-14,2019-10-17,2019-10-17,JAWA TENGAH,KEBUMEN,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RITP,PBI APBN,A01 Typhoid and paratyphoid fevers,A01,A01,Typhoid and paratyphoid fevers,Rawat Inap di R. Perawatan Biasa,600000,600000
5,91875907,43917593,3.267601,161901019P002249,2019-10-15,2019-10-18,2019-10-18,JAWA TENGAH,SUKOHARJO,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RITP,PBI APBN,A01 Typhoid and paratyphoid fevers,A01,A010,Typhoid fever,Rawat Inap di R. Perawatan Biasa,600000,600000
6,66909948,13554633,15.368465,207870516Y000998,2016-05-21,2016-05-25,2016-05-25,LAMPUNG,LAMPUNG TENGAH,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RITP,PBI APBN,A01 Typhoid and paratyphoid fevers,A01,A010,Typhoid fever,Rawat Inap di R. Perawatan Biasa,480000,480000
7,298813915,405847894,2.459305,257830718Y001284,2018-07-16,2018-07-19,2018-07-19,SULAWESI SELATAN,GOWA,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RITP,PPU,A01 Typhoid and paratyphoid fevers,A01,A01,Typhoid and paratyphoid fevers,Rawat Inap di R. Perawatan Biasa,360000,360000
8,392983246,392983246,7.131634,277311018Y000005,2018-10-01,2018-10-01,2018-10-01,SULAWESI SELATAN,SOPPENG,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RJTP,PBI APBN,A01 Typhoid and paratyphoid fevers,A01,A01,Typhoid and paratyphoid fevers,Evakuasi medis / Ambulans Darat,148824,148824
9,81350856,81350856,18.080889,63910215Y000060,2015-02-21,2015-02-21,2015-02-21,ACEH,BIREUEN,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RITP,PBI APBD,A01 Typhoid and paratyphoid fevers,A01,A010,Typhoid fever,Evakuasi medis / Ambulans Darat,68000,68000




————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————


✅ Semua data berhasil dibersihkan dan disimpan tanpa kolom Unnamed.


### **1.3 Penyamaan Struktur Primary Table dan Related Table Berdasarkan "PSTV01" (no_peserta)**
Dilakukan penyamaan jumlah baris antara primary table dan related table karena hasil inspeksi menunjukkan perbedaan signifikan, yaitu 94.966 baris pada related table dan 36.493 baris pada primary table. Perbedaan ini menunjukkan adanya "PSTV01" (no_peserta) yang tidak saling terhubung di antara kedua tabel. Oleh karena itu, dilakukan pencocokan terhadap kolom "PSTV01" (no_peserta) dan penghapusan data yang tidak memiliki pasangan di kedua tabel agar hanya tersisa entri yang saling berelasi.

---

In [ ]:
# -----------------------------------------------------------------------------
# Penyamaan Struktur Primary Table dan Related Table Berdasarkan "PSTV01" (no_peserta)
# -----------------------------------------------------------------------------

# Dictionary file paths clean
file_paths_clean = {
    "Kepesertaan": clean_file_paths["Kepesertaan"],
    "Non Kapitasi": clean_file_paths["Non Kapitasi"],
}

# Load file csv
df_kepesertaan = pd.read_csv(file_paths_clean["Kepesertaan"])
df_nonkapitasi = pd.read_csv(file_paths_clean["Non Kapitasi"])

# === Pastikan kolom 'PSTV01' ada di kedua tabel ===
if 'PSTV01' not in df_kepesertaan.columns or 'PSTV01' not in df_nonkapitasi.columns:
    raise ValueError("⚠️ Kolom 'PSTV01' tidak ditemukan di salah satu tabel!")

# === Hanya pertahankan peserta yang ada di kedua tabel ===
common_ids = df_kepesertaan['PSTV01'].isin(df_nonkapitasi['PSTV01'])
df_kepesertaan_common = df_kepesertaan[common_ids]

common_ids_non = df_nonkapitasi['PSTV01'].isin(df_kepesertaan['PSTV01'])
df_nonkapitasi_common = df_nonkapitasi[common_ids_non]

print("Jumlah peserta yang sama di kedua tabel:", len(df_kepesertaan_common))

# === Urutkan kedua tabel berdasarkan 'PSTV01' agar baris sesuai ===
df_kepesertaan_common_sorted = df_kepesertaan_common.sort_values('PSTV01').reset_index(drop=True)
df_nonkapitasi_common_sorted = df_nonkapitasi_common.sort_values('PSTV01').reset_index(drop=True)

# === Display tabel (misal 10 baris pertama) ===
print("=== df_kepesertaan ===")
display(df_kepesertaan_common_sorted.head(10))

print("\n")
print("—" * 500)
print("\n")

print("=== df_nonkapitasi ===")
display(df_nonkapitasi_common_sorted.head(10))

print("✅ Kedua tabel telah disamakan dan diurutkan berdasarkan PSTV01.")


Jumlah peserta yang sama di kedua tabel: 8328
=== df_kepesertaan ===


,PSTV01,PSTV02,PSTV03,PSTV04,PSTV05,PSTV06,PSTV07,PSTV08,PSTV09,PSTV10,PSTV11,PSTV12,PSTV13,PSTV14,PSTV15,PSTV16,PSTV17,PSTV18
0,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,KELAS II,PPU,RIAU,INDRAGIRI HULU,SWASTA,KLINIK PRATAMA,RIAU,PELALAWAN,12.892989,2021,MENINGGAL,2019.0
1,66027,65799350,1945-12-31,ISTRI,PEREMPUAN,KAWIN,KELAS III,PBI APBN,SULAWESI SELATAN,SINJAI,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,SULAWESI SELATAN,SINJAI,18.593020,2021,MENINGGAL,2021.0
2,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,KELAS III,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN
3,88912,62348134,1995-08-20,ANAK,PEREMPUAN,BELUM KAWIN,KELAS III,PBI APBN,JAWA BARAT,INDRAMAYU,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,JAWA BARAT,SUMEDANG,1.797199,2021,AKTIF,NaN
4,133712,133712,1977-11-02,PESERTA,PEREMPUAN,CERAI,KELAS III,PBI APBN,ACEH,BENER MERIAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,BENER MERIAH,2.066524,2021,AKTIF,NaN
5,184922,184922,1960-04-01,PESERTA,PEREMPUAN,KAWIN,KELAS III,PBPU,JAWA TIMUR,MALANG,TNI AD,KLINIK PRATAMA,JAWA TIMUR,MALANG,15.680199,2021,MENINGGAL,2021.0
6,191047,17570556,2001-08-21,TAMBAHAN,PEREMPUAN,KAWIN,KELAS III,PBI APBN,JAWA TENGAH,REMBANG,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,JAWA TENGAH,REMBANG,1.054065,2021,AKTIF,NaN
7,216422,216422,1943-03-03,PESERTA,PEREMPUAN,CERAI,KELAS II,BUKAN PEKERJA,SULAWESI SELATAN,LUWU UTARA,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,SULAWESI SELATAN,LUWU UTARA,7.640796,2021,AKTIF,NaN
8,225267,225267,1968-05-10,PESERTA,PEREMPUAN,KAWIN,KELAS III,PBI APBN,NUSA TENGGARA BARAT,DOMPU,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NUSA TENGGARA BARAT,DOMPU,7.693568,2021,AKTIF,NaN
9,235632,47609425,2001-02-07,ANAK,PEREMPUAN,BELUM KAWIN,KELAS III,PBPU,RIAU,SIAK,SWASTA,KLINIK PRATAMA,RIAU,SIAK,6.394610,2021,AKTIF,NaN




————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————


=== df_nonkapitasi ===


,PSTV01,PSTV02,PSTV15,PNK02,PNK03,PNK04,PNK05,PNK06,PNK07,PNK08,PNK09,PNK10,PNK11,PNK12,PNK13,PNK13A,PNK14,PNK15,PNK16,PNK17,PNK18
0,65638,81863275,12.892989,446150419P001467,2019-04-10,2019-04-10,2019-04-10,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,PPU,"A15 Respiratory tuberculosis, bacteriologicall...",A15,A150,"Tuberculosis of lung, confirmed by sputum micr...",Evakuasi medis / Ambulans Darat,480000,480000
1,65638,81863275,12.892989,446150519P001161,2019-05-10,2019-05-10,2019-05-10,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,PPU,R10 Abdominal and pelvic pain,R10,R100,Acute abdomen,Evakuasi medis / Ambulans Darat,480000,480000
2,65638,81863275,12.892989,446270518Y000034,2018-05-06,2018-05-06,2018-05-06,RIAU,PELALAWAN,SWASTA,DOKTER UMUM,DOKTER PRAKTER PERORANGAN,RJTP,PPU,H33 Retinal detachments and breaks,H33,H335,Other retinal detachments,Evakuasi medis / Ambulans Darat,580000,580000
3,65638,81863275,12.892989,446150219P000798,2019-02-16,2019-02-16,2019-02-16,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,PPU,"A15 Respiratory tuberculosis, bacteriologicall...",A15,A150,"Tuberculosis of lung, confirmed by sputum micr...",Evakuasi medis / Ambulans Darat,480000,480000
4,65638,81863275,12.892989,446150619P001349,2019-06-07,2019-06-07,2019-06-07,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,PPU,R10 Abdominal and pelvic pain,R10,R10,Abdominal and pelvic pain,Evakuasi medis / Ambulans Darat,480000,480000
5,66027,65799350,18.593020,276980416Y000130,2016-04-21,2016-04-21,2016-04-28,SULAWESI SELATAN,SINJAI,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RITP,PBI APBN,D50 Iron deficiency anaemia,D50,D509,"Iron deficiency anaemia, unspecified",Evakuasi medis / Ambulans Darat,52500,52500
6,68930,68930,1.627409,105980618Y000955,2018-06-24,2018-06-24,2018-06-24,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RJTP,PBI APBD,Z39 Postpartum care and examination,Z39,Z392,Routine postpartum follow-up,Pelayanan PNC 2 (Dua),25000,25000
7,68930,68930,1.627409,105980618Y001023,2018-06-21,2018-06-21,2018-06-21,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RITP,PBI APBD,O80 Single spontaneous delivery,O80,O808,Other single spontaneous delivery,Paket Persalinan per Vaginam normal (oleh Bidan),700000,700000
8,68930,68930,1.627409,105980618Y000954,2018-06-22,2018-06-22,2018-06-22,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RJTP,PBI APBD,Z39 Postpartum care and examination,Z39,Z392,Routine postpartum follow-up,Pelayanan PNC 1 (Satu),25000,25000
9,68930,68930,1.627409,105980718Y001262,2018-07-31,2018-07-31,2018-07-31,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RJTP,PBI APBD,Z39 Postpartum care and examination,Z39,Z392,Routine postpartum follow-up,Pelayanan PNC 4 (Empat),25000,25000


✅ Kedua tabel telah disamakan dan diurutkan berdasarkan PSTV01.


### **1.4 Normalisasi Penamaan dan Penyelarasan Atribut Kolom**
Pada tahap ini dilakukan normalisasi penamaan atribut karena kolom pada primary table dan related table masih menggunakan kode yang sulit dibaca. Untuk memudahkan interpretasi dan analisis, setiap atribut diganti dengan nama yang lebih deskriptif.

Selain penyeragaman nama, dilakukan juga transformasi tambahan berupa perubahan seluruh nama kolom menjadi huruf kecil, penggantian spasi dengan garis bawah (underscore). Langkah ini bertujuan untuk meningkatkan keterbacaan dan konsistensi struktur data selama proses analisis.

---

In [ ]:
# -----------------------------------------------------------------------------
# Normalisasi Penamaan dan Penyelarasan Atribut Kolom
# -----------------------------------------------------------------------------

# Ganti nama kolom tertentu
df_kepesertaan_common_sorted.rename(columns={
    "PSTV01": "No_pEserta",
    "PSTV02": "nO_keluarga",
    "PSTV03": "TGL_Lahir",
    "PSTV04": "hub_keluarga",
    "PSTV05": "gender",
    "PSTV06": "status_pernikahan",
    "PSTV07": "kelas_rawat",
    "PSTV08": "segmen_peserta",
    "PSTV09": "prov_peserta",
    "PSTV10": "kab_kota_peserta",
    "PSTV11": "kepemilikan_faskes",
    "PSTV12": "jenis_faskes",
    "PSTV13": "prov_faskes",
    "PSTV14": "kab_kota_faskes",
    "PSTV15": "bobot",
    "PSTV16": "Tahun_Sampel",
    "PSTV17": "status_kepesertaan",
    "PSTV18": "thn_meninggal"
}, inplace=True)

df_nonkapitasi_common_sorted.rename(columns={
    "PSTV01": "No_Peserta",
    "PSTV02": "no_keluarga",
    "PSTV15": "bobot",
    "PNK02": "id_kunjungan",
    "PNK03": "tgl_kunjungan",
    "PNK04": "tgl_tindakan",
    "PNK05": "tgl_pulang",
    "PNK06": "prov_faskes",
    "PNK07": "kab_kota_faskes",
    "PNK08": "kepemilikan_faskes",
    "PNK09": "jenis_faskes",
    "PNK10": "tipe_faskes",
    "PNK11": "tingkat_layanan",
    "PNK12": "segmen_peserta",
    "PNK13": "kode_nama_diagnosis_ICD10",
    "PNK13A": "kode_diagnosis_ICD10",
    "PNK14": "kode_diagnosis",
    "PNK15": "nama_diagnosis",
    "PNK16": "nama_tindakan",
    "PNK17": "biaya_tagihan",
    "PNK18": "biaya_verifikasi"
}, inplace=True)

print("✅ Nama kolom di kedua tabel berhasil diubah.")

# Ganti semua kolom jadi huruf kecil dan ganti spasi dengan underscore
df_kepesertaan_common_sorted.columns = (
    df_kepesertaan_common_sorted.columns
    .str.lower()
    .str.replace(" ", "_")
)

df_nonkapitasi_common_sorted.columns = (
    df_nonkapitasi_common_sorted.columns
    .str.lower()
    .str.replace(" ", "_")
)

print("✅ Semua nama kolom diubah jadi huruf kecil tanpa spasi.")


✅ Nama kolom di kedua tabel berhasil diubah.
✅ Semua nama kolom diubah jadi huruf kecil tanpa spasi.


### **1.5 Penggabungan Tabel, Penyeragaman Kolom Duplikat, dan Penataan Struktur Data**

Pada tahap ini dilakukan penggabungan primary table dan related table menggunakan no_peserta sebagai key join untuk memastikan kesesuaian data antar tabel. Setelah proses penyamaan baris berdasarkan no_peserta, ditemukan beberapa pasangan kolom dengan isi yang identik namun memiliki nama atau kode berbeda. Kolom-kolom tersebut dikategorikan sebagai duplikat dan diseragamkan, seperti:

* PSTV08 = PNK12 → segmen_peserta
* PSTV11 = PNK08 → kepemilikan_faskes
* PSTV12 = PNK09 → jenis_faskes
* PSTV13 = PNK06 → prov_faskes
* PSTV14 = PNK07 → kab_kota_faskes

Setelah kolom duplikat dihapus dan penamaan diseragamkan, dilakukan penataan ulang urutan kolom agar struktur dataset menjadi lebih teratur, mudah dibaca, dan siap untuk tahap pra-pemrosesan berikutnya.

---

In [ ]:
# -----------------------------------------------------------------------------
# Penggabungan Tabel, Penyeragaman Kolom Duplikat, dan Penataan Struktur Data
# -----------------------------------------------------------------------------

# === Cek kolom dan join ===
if "no_peserta" not in df_kepesertaan_common_sorted.columns or "no_peserta" not in df_nonkapitasi_common_sorted.columns:
    raise ValueError("Kolom 'no_peserta' tidak ditemukan di salah satu tabel!")

common_cols = set(df_kepesertaan_common_sorted.columns).intersection(df_nonkapitasi_common_sorted.columns)
common_cols.discard("no_peserta")
df_nonkapitasi_nodup = df_nonkapitasi_common_sorted.drop(columns=common_cols)

df_gabungan = pd.merge(
    df_kepesertaan_common_sorted,
    df_nonkapitasi_nodup,
    on="no_peserta",
    how="inner"
)

print("✅ Penggabungan berhasil!")
print(f"Jumlah baris hasil gabungan: {len(df_gabungan)}")
print(f"Jumlah kolom hasil gabungan: {len(df_gabungan.columns)}")
display(df_gabungan.head(10))

print("\n")
print("—" * 500)
print("\n")

# === Tentukan urutan kolom yang kamu inginkan ===
urutan_kolom = [
    "no_peserta", "no_keluarga", "tgl_lahir", "hub_keluarga", "gender",
    "status_pernikahan", "segmen_peserta", "prov_peserta", "kab_kota_peserta",
    "bobot", "tahun_sampel", "status_kepesertaan", "thn_meninggal",
    "id_kunjungan", "tgl_kunjungan", "tgl_tindakan", "tgl_pulang",
    "kelas_rawat", "prov_faskes", "kab_kota_faskes", "kepemilikan_faskes",
    "jenis_faskes", "tipe_faskes", "tingkat_layanan",
    "kode_nama_diagnosis_icd10", "kode_diagnosis_icd10", "kode_diagnosis",
    "nama_diagnosis", "nama_tindakan", "biaya_tagihan", "biaya_verifikasi"
]

# === Cek kolom mana yang ada di tabel gabungan ===
kolom_ada = [kol for kol in urutan_kolom if kol in df_gabungan.columns]
kolom_tidak_ada = [kol for kol in urutan_kolom if kol not in df_gabungan.columns]

# === Susun ulang kolom berdasarkan urutan yang ada ===
df_gabungan_reordered = df_gabungan[kolom_ada]

# === (Opsional) Tambahkan info kolom yang tidak ditemukan ===
if kolom_tidak_ada:
    print("⚠️ Kolom berikut tidak ditemukan di tabel gabungan:")
    for kol in kolom_tidak_ada:
        print(f" - {kol}")
else:
    print("✅ Semua kolom ditemukan dan telah diurutkan sesuai urutan yang diinginkan.")

# === Tampilkan hasil ===
display(df_gabungan_reordered.head(10))

✅ Penggabungan berhasil!
Jumlah baris hasil gabungan: 36493
Jumlah kolom hasil gabungan: 31


,no_peserta,no_keluarga,tgl_lahir,hub_keluarga,gender,status_pernikahan,kelas_rawat,segmen_peserta,prov_peserta,kab_kota_peserta,kepemilikan_faskes,jenis_faskes,prov_faskes,kab_kota_faskes,bobot,tahun_sampel,status_kepesertaan,thn_meninggal,id_kunjungan,tgl_kunjungan,tgl_tindakan,tgl_pulang,tipe_faskes,tingkat_layanan,kode_nama_diagnosis_icd10,kode_diagnosis_icd10,kode_diagnosis,nama_diagnosis,nama_tindakan,biaya_tagihan,biaya_verifikasi
0,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,KELAS II,PPU,RIAU,INDRAGIRI HULU,SWASTA,KLINIK PRATAMA,RIAU,PELALAWAN,12.892989,2021,MENINGGAL,2019.0,446150419P001467,2019-04-10,2019-04-10,2019-04-10,KLINIK RAWAT INAP,RJTP,"A15 Respiratory tuberculosis, bacteriologicall...",A15,A150,"Tuberculosis of lung, confirmed by sputum micr...",Evakuasi medis / Ambulans Darat,480000,480000
1,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,KELAS II,PPU,RIAU,INDRAGIRI HULU,SWASTA,KLINIK PRATAMA,RIAU,PELALAWAN,12.892989,2021,MENINGGAL,2019.0,446150519P001161,2019-05-10,2019-05-10,2019-05-10,KLINIK RAWAT INAP,RJTP,R10 Abdominal and pelvic pain,R10,R100,Acute abdomen,Evakuasi medis / Ambulans Darat,480000,480000
2,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,KELAS II,PPU,RIAU,INDRAGIRI HULU,SWASTA,KLINIK PRATAMA,RIAU,PELALAWAN,12.892989,2021,MENINGGAL,2019.0,446270518Y000034,2018-05-06,2018-05-06,2018-05-06,DOKTER PRAKTER PERORANGAN,RJTP,H33 Retinal detachments and breaks,H33,H335,Other retinal detachments,Evakuasi medis / Ambulans Darat,580000,580000
3,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,KELAS II,PPU,RIAU,INDRAGIRI HULU,SWASTA,KLINIK PRATAMA,RIAU,PELALAWAN,12.892989,2021,MENINGGAL,2019.0,446150219P000798,2019-02-16,2019-02-16,2019-02-16,KLINIK RAWAT INAP,RJTP,"A15 Respiratory tuberculosis, bacteriologicall...",A15,A150,"Tuberculosis of lung, confirmed by sputum micr...",Evakuasi medis / Ambulans Darat,480000,480000
4,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,KELAS II,PPU,RIAU,INDRAGIRI HULU,SWASTA,KLINIK PRATAMA,RIAU,PELALAWAN,12.892989,2021,MENINGGAL,2019.0,446150619P001349,2019-06-07,2019-06-07,2019-06-07,KLINIK RAWAT INAP,RJTP,R10 Abdominal and pelvic pain,R10,R10,Abdominal and pelvic pain,Evakuasi medis / Ambulans Darat,480000,480000
5,66027,65799350,1945-12-31,ISTRI,PEREMPUAN,KAWIN,KELAS III,PBI APBN,SULAWESI SELATAN,SINJAI,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,SULAWESI SELATAN,SINJAI,18.593020,2021,MENINGGAL,2021.0,276980416Y000130,2016-04-21,2016-04-21,2016-04-28,RAWAT INAP,RITP,D50 Iron deficiency anaemia,D50,D509,"Iron deficiency anaemia, unspecified",Evakuasi medis / Ambulans Darat,52500,52500
6,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,KELAS III,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980618Y000955,2018-06-24,2018-06-24,2018-06-24,NON RAWAT INAP,RJTP,Z39 Postpartum care and examination,Z39,Z392,Routine postpartum follow-up,Pelayanan PNC 2 (Dua),25000,25000
7,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,KELAS III,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980618Y001023,2018-06-21,2018-06-21,2018-06-21,NON RAWAT INAP,RITP,O80 Single spontaneous delivery,O80,O808,Other single spontaneous delivery,Paket Persalinan per Vaginam normal (oleh Bidan),700000,700000
8,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,KELAS III,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980618Y000954,2018-06-22,2018-06-22,2018-06-22,NON RAWAT INAP,RJTP,Z39 Postpartum care and examination,Z39,Z392,Routine postpartum follow-up,Pelayanan PNC 1 (Satu),25000,25000
9,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,KELAS III,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980718Y0012



————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————


✅ Semua kolom ditemukan dan telah diurutkan sesuai urutan yang diinginkan.


,no_peserta,no_keluarga,tgl_lahir,hub_keluarga,gender,status_pernikahan,segmen_peserta,prov_peserta,kab_kota_peserta,bobot,tahun_sampel,status_kepesertaan,thn_meninggal,id_kunjungan,tgl_kunjungan,tgl_tindakan,tgl_pulang,kelas_rawat,prov_faskes,kab_kota_faskes,kepemilikan_faskes,jenis_faskes,tipe_faskes,tingkat_layanan,kode_nama_diagnosis_icd10,kode_diagnosis_icd10,kode_diagnosis,nama_diagnosis,nama_tindakan,biaya_tagihan,biaya_verifikasi
0,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,2021,MENINGGAL,2019.0,446150419P001467,2019-04-10,2019-04-10,2019-04-10,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,"A15 Respiratory tuberculosis, bacteriologicall...",A15,A150,"Tuberculosis of lung, confirmed by sputum micr...",Evakuasi medis / Ambulans Darat,480000,480000
1,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,2021,MENINGGAL,2019.0,446150519P001161,2019-05-10,2019-05-10,2019-05-10,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,R10 Abdominal and pelvic pain,R10,R100,Acute abdomen,Evakuasi medis / Ambulans Darat,480000,480000
2,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,2021,MENINGGAL,2019.0,446270518Y000034,2018-05-06,2018-05-06,2018-05-06,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,DOKTER PRAKTER PERORANGAN,RJTP,H33 Retinal detachments and breaks,H33,H335,Other retinal detachments,Evakuasi medis / Ambulans Darat,580000,580000
3,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,2021,MENINGGAL,2019.0,446150219P000798,2019-02-16,2019-02-16,2019-02-16,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,"A15 Respiratory tuberculosis, bacteriologicall...",A15,A150,"Tuberculosis of lung, confirmed by sputum micr...",Evakuasi medis / Ambulans Darat,480000,480000
4,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,2021,MENINGGAL,2019.0,446150619P001349,2019-06-07,2019-06-07,2019-06-07,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,R10 Abdominal and pelvic pain,R10,R10,Abdominal and pelvic pain,Evakuasi medis / Ambulans Darat,480000,480000
5,66027,65799350,1945-12-31,ISTRI,PEREMPUAN,KAWIN,PBI APBN,SULAWESI SELATAN,SINJAI,18.593020,2021,MENINGGAL,2021.0,276980416Y000130,2016-04-21,2016-04-21,2016-04-28,KELAS III,SULAWESI SELATAN,SINJAI,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RITP,D50 Iron deficiency anaemia,D50,D509,"Iron deficiency anaemia, unspecified",Evakuasi medis / Ambulans Darat,52500,52500
6,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980618Y000955,2018-06-24,2018-06-24,2018-06-24,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RJTP,Z39 Postpartum care and examination,Z39,Z392,Routine postpartum follow-up,Pelayanan PNC 2 (Dua),25000,25000
7,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980618Y001023,2018-06-21,2018-06-21,2018-06-21,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RITP,O80 Single spontaneous delivery,O80,O808,Other single spontaneous delivery,Paket Persalinan per Vaginam normal (oleh Bidan),700000,700000
8,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980618Y000954,2018-06-22,2018-06-22,2018-06-22,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RJTP,Z39 Postpartum care and examination,Z39,Z392,Routine postpartum follow-up,Pelayanan PNC 1 (Satu),25000,25000
9,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980718Y001262,2018-07-31,2018-07-31,2018-07-31,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,

## **2. Data Preprocessing**
Data preprocessing adalah tahap persiapan data agar bersih, konsisten, dan siap untuk pemodelan. Pada tahap ini dilakukan pemeriksaan dan penanganan missing value, identifikasi serta penanganan outlier, pembuatan fitur baru seperti usia_kunjungan dan lama_rawat, serta analisis korelasi numerik dan kategorikal terhadap target PNK17 (biaya_tagihan).

---

### **2.1 Cek Missing Value**
Cek missing value dilakukan untuk mengetahui apakah ada data yang hilang di dataset dan seberapa banyak pengaruhnya terhadap analisis atau pemodelan.

Panduan Penanganan Missing Value:
* Hapus baris: jika missing <5% dan kolom penting.

* Imputasi/Capping: jika missing >5–30%, isi dengan mean, median, modus, atau kategori “Unknown”.
* Hapus kolom: jika missing >30–40% atau kolom kurang penting.

*Catatan: Threshold bisa disesuaikan dengan ukuran dataset dan relevansi kolom.*

---

In [ ]:
# -----------------------------------------------------------------------------
# Cek missing value
# -----------------------------------------------------------------------------

def cek_missing(df, nama_tabel):
    print(f"\n=== Cek Missing Value: {nama_tabel} ===")
    missing_count = df.isnull().sum()
    missing_percent = (missing_count / len(df)) * 100
    missing_df = pd.DataFrame({
        'Jumlah Missing': missing_count,
        'Persentase (%)': missing_percent
    })
    display(missing_df[missing_df['Jumlah Missing'] > 0])

# === Jalankan fungsi cek missing ===
cek_missing(df_gabungan, "df_gabungan")


=== Cek Missing Value: df_gabungan ===


,Jumlah Missing,Persentase (%)
thn_meninggal,32829,89.959718
kode_nama_diagnosis_icd10,2,0.005481
kode_diagnosis_icd10,2,0.005481


### **2.2 Penanganan Missing Value**

Dari pengecekan, ditemukan missing value pada kolom kode_nama_diagnosis_icd10 dan kode_diagnosis_icd10 masing-masing sebanyak 2 baris (0,0055%). Sedangkan kolom thn_meninggal memiliki 32.829 missing value (89,96%). Untuk kolom thn_meninggal, kondisi ini wajar dan dibiarkan karena memang tidak semua peserta memiliki data kematian. Sementara itu, missing value yang sangat sedikit pada kolom diagnosis (kurang dari 5%) dihapus karena tidak signifikan dan tidak mempengaruhi analisis.

---

In [ ]:
# -----------------------------------------------------------------------------
# Penanganan Missing Value
# -----------------------------------------------------------------------------

# === Hapus baris yang memiliki missing value di kolom 'kode_nama_diagnosis_icd10' dan 'kode_diagnosis_icd10' ===
kolom_target = ['kode_nama_diagnosis_icd10', 'kode_diagnosis_icd10']

# Tampilkan jumlah missing sebelum dihapus
print("Jumlah missing sebelum dihapus:")
display(df_gabungan_reordered[kolom_target].isnull().sum())

# Hapus baris yang memiliki missing di salah satu kolom tersebut
df_gabungan_reordered_clean = df_gabungan_reordered.dropna(subset=kolom_target)

# Tampilkan hasil setelah dihapus
print("\n✅ Baris dengan missing value di kode_nama_diagnosis_icd10 dan kode_diagnosis_icd10 sudah dihapus.")
print(f"Jumlah baris sebelum: {len(df_gabungan_reordered)}")
print(f"Jumlah baris sesudah: {len(df_gabungan_reordered_clean)}")

# Cek ulang apakah masih ada missing
print("\nCek ulang missing setelah penghapusan:")
display(df_gabungan_reordered_clean[kolom_target].isnull().sum())

# Opsional: tampilkan beberapa baris hasil akhir
display(df_gabungan_reordered_clean.head(10))

Jumlah missing sebelum dihapus:


,0
kode_nama_diagnosis_icd10,2
kode_diagnosis_icd10,2



✅ Baris dengan missing value di kode_nama_diagnosis_icd10 dan kode_diagnosis_icd10 sudah dihapus.
Jumlah baris sebelum: 36493
Jumlah baris sesudah: 36491

Cek ulang missing setelah penghapusan:


,0
kode_nama_diagnosis_icd10,0
kode_diagnosis_icd10,0


,no_peserta,no_keluarga,tgl_lahir,hub_keluarga,gender,status_pernikahan,segmen_peserta,prov_peserta,kab_kota_peserta,bobot,tahun_sampel,status_kepesertaan,thn_meninggal,id_kunjungan,tgl_kunjungan,tgl_tindakan,tgl_pulang,kelas_rawat,prov_faskes,kab_kota_faskes,kepemilikan_faskes,jenis_faskes,tipe_faskes,tingkat_layanan,kode_nama_diagnosis_icd10,kode_diagnosis_icd10,kode_diagnosis,nama_diagnosis,nama_tindakan,biaya_tagihan,biaya_verifikasi
0,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,2021,MENINGGAL,2019.0,446150419P001467,2019-04-10,2019-04-10,2019-04-10,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,"A15 Respiratory tuberculosis, bacteriologicall...",A15,A150,"Tuberculosis of lung, confirmed by sputum micr...",Evakuasi medis / Ambulans Darat,480000,480000
1,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,2021,MENINGGAL,2019.0,446150519P001161,2019-05-10,2019-05-10,2019-05-10,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,R10 Abdominal and pelvic pain,R10,R100,Acute abdomen,Evakuasi medis / Ambulans Darat,480000,480000
2,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,2021,MENINGGAL,2019.0,446270518Y000034,2018-05-06,2018-05-06,2018-05-06,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,DOKTER PRAKTER PERORANGAN,RJTP,H33 Retinal detachments and breaks,H33,H335,Other retinal detachments,Evakuasi medis / Ambulans Darat,580000,580000
3,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,2021,MENINGGAL,2019.0,446150219P000798,2019-02-16,2019-02-16,2019-02-16,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,"A15 Respiratory tuberculosis, bacteriologicall...",A15,A150,"Tuberculosis of lung, confirmed by sputum micr...",Evakuasi medis / Ambulans Darat,480000,480000
4,65638,81863275,1985-01-20,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,2021,MENINGGAL,2019.0,446150619P001349,2019-06-07,2019-06-07,2019-06-07,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,R10 Abdominal and pelvic pain,R10,R10,Abdominal and pelvic pain,Evakuasi medis / Ambulans Darat,480000,480000
5,66027,65799350,1945-12-31,ISTRI,PEREMPUAN,KAWIN,PBI APBN,SULAWESI SELATAN,SINJAI,18.593020,2021,MENINGGAL,2021.0,276980416Y000130,2016-04-21,2016-04-21,2016-04-28,KELAS III,SULAWESI SELATAN,SINJAI,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RITP,D50 Iron deficiency anaemia,D50,D509,"Iron deficiency anaemia, unspecified",Evakuasi medis / Ambulans Darat,52500,52500
6,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980618Y000955,2018-06-24,2018-06-24,2018-06-24,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RJTP,Z39 Postpartum care and examination,Z39,Z392,Routine postpartum follow-up,Pelayanan PNC 2 (Dua),25000,25000
7,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980618Y001023,2018-06-21,2018-06-21,2018-06-21,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RITP,O80 Single spontaneous delivery,O80,O808,Other single spontaneous delivery,Paket Persalinan per Vaginam normal (oleh Bidan),700000,700000
8,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980618Y000954,2018-06-22,2018-06-22,2018-06-22,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RJTP,Z39 Postpartum care and examination,Z39,Z392,Routine postpartum follow-up,Pelayanan PNC 1 (Satu),25000,25000
9,68930,68930,1984-10-17,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,2021,AKTIF,NaN,105980718Y001262,2018-07-31,2018-07-31,2018-07-31,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,

### **2.3 Pembuatan Fitur Baru: usia_pasien dan durasi_rawat**

Kita akan membuat fitur baru yang berfungsi sebagai prediktor untuk model dengan menggunakan kolom tgl_lahir, tgl_kunjungan, dan tgl_pulang. Fitur pertama adalah usia_pasien, yaitu selisih antara tgl_kunjungan dan tgl_lahir, yang menunjukkan usia pasien saat mengunjungi Fasilitas Kesehatan Tingkat Pertama (FKTP). Fitur kedua adalah durasi_rawat, yaitu selisih antara tgl_pulang dan tgl_kunjungan, untuk mengetahui durasi perawatan pasien. Fitur-fitur baru ini relevan dan diharapkan dapat meningkatkan akurasi prediksi biaya_tagihan.

---
Kolom-kolom berikut dieksklusi karena dianggap tidak relevan untuk prediksi biaya_tagihan. Sementara itu, kolom kode_diagnosis_icd10, kode_nama_diagnosis_icd10, dan kode_diagnosis tidak disertakan karena informasi diagnosis telah terwakili oleh kolom nama_diagnosis:

['tgl_lahir', 'tgl_kunjungan', 'tgl_tindakan', 'tgl_pulang', 'kode_diagnosis', 'kode_diagnosis_icd10', 'kode_nama_diagnosis_icd10', 'tahun_sampel', 'status_kepesertaan', 'thn_meninggal', 'id_kunjungan', 'biaya_verifikasi']

Selain itu, kolom no_peserta dan no_keluarga tidak digunakan dalam proses training, testing, maupun modeling, melainkan hanya ditampilkan untuk keperluan verifikasi data peserta.

---

In [ ]:
# -----------------------------------------------------------------------------
# Pembuatan Fitur Baru: usia_pasien dan durasi_rawat
# -----------------------------------------------------------------------------

# Pastikan kolom tanggal dikonversi ke datetime dulu
df_gabungan_reordered_clean['tgl_lahir'] = pd.to_datetime(df_gabungan_reordered_clean['tgl_lahir'], errors='coerce')
df_gabungan_reordered_clean['tgl_kunjungan'] = pd.to_datetime(df_gabungan_reordered_clean['tgl_kunjungan'], errors='coerce')
df_gabungan_reordered_clean['tgl_pulang'] = pd.to_datetime(df_gabungan_reordered_clean['tgl_pulang'], errors='coerce')

# === Buat fitur usia_pasien (dalam tahun) ===
df_gabungan_reordered_clean['usia_pasien'] = (
    (df_gabungan_reordered_clean['tgl_kunjungan'] - df_gabungan_reordered_clean['tgl_lahir'])
    .dt.days / 365.25
).round(1)

# === Buat fitur durasi_rawat (dalam hari) ===
df_gabungan_reordered_clean['durasi_rawat'] = (
    (df_gabungan_reordered_clean['tgl_pulang'] - df_gabungan_reordered_clean['tgl_kunjungan'])
    .dt.days
)

# === Cek hasilnya ===
print("Contoh hasil kolom baru:")
display(df_gabungan_reordered_clean[['tgl_lahir', 'tgl_kunjungan', 'tgl_pulang', 'usia_pasien', 'durasi_rawat']].head(10))

print("\n")
print("—" * 500)
print("\n")

# Tentukan daftar kolom yang ingin Anda buang
kolom_untuk_dibuang = [
    'tgl_lahir',
    'tgl_kunjungan',
    'tgl_tindakan',
    'tgl_pulang',
    'kode_diagnosis',
    'kode_nama_diagnosis_icd10',
    'tahun_sampel',
    'status_kepesertaan',
    'thn_meninggal',
    #'no_peserta',
    #'no_keluarga',
    'id_kunjungan',
    'biaya_verifikasi',
    'kode_diagnosis_icd10'
]

df_gabungan_reordered_clean_clean = df_gabungan_reordered_clean.drop(
    columns=kolom_untuk_dibuang,
    errors='ignore'
)

# === Cek DataFrame BARU ('_clean_clean') ===
print("DataFrame baru 'df_gabungan_reordered_clean_clean' telah dibuat.")
print(f"Jumlah kolom di DataFrame LAMA: {df_gabungan_reordered_clean.shape[1]}")
print(f"Jumlah kolom di DataFrame BARU: {df_gabungan_reordered_clean_clean.shape[1]}\n")

memory_bytes = df_gabungan_reordered_clean_clean.memory_usage(deep=True).sum()
memory_mb = memory_bytes / (1024 ** 2)

print(f"Jumlah baris (baru): {df_gabungan_reordered_clean_clean.shape[0]}")
print(f"Jumlah kolom (baru): {df_gabungan_reordered_clean_clean.shape[1]}")
print(f"Ukuran di memori (baru): {memory_mb:.2f} MB\n")

print("Tipe data setiap kolom (DataFrame BARU):")
print(df_gabungan_reordered_clean_clean.dtypes)

print("Tabel yang sudah siap untuk preprocessing selanjutnya")
display(df_gabungan_reordered_clean_clean[['no_peserta', 'no_keluarga', 'usia_pasien', 'hub_keluarga', 'gender', 'status_pernikahan',
                                           'segmen_peserta', 'prov_peserta', 'kab_kota_peserta', 'bobot', 'durasi_rawat', 'kelas_rawat',
                                           'prov_faskes', 'kab_kota_faskes', 'kepemilikan_faskes', 'jenis_faskes', 'tipe_faskes',
                                           'tingkat_layanan', 'nama_diagnosis', 'nama_tindakan', 'biaya_tagihan']].head(10))

Contoh hasil kolom baru:


,tgl_lahir,tgl_kunjungan,tgl_pulang,usia_pasien,durasi_rawat
0,1985-01-20,2019-04-10,2019-04-10,34.2,0
1,1985-01-20,2019-05-10,2019-05-10,34.3,0
2,1985-01-20,2018-05-06,2018-05-06,33.3,0
3,1985-01-20,2019-02-16,2019-02-16,34.1,0
4,1985-01-20,2019-06-07,2019-06-07,34.4,0
5,1945-12-31,2016-04-21,2016-04-28,70.3,7
6,1984-10-17,2018-06-24,2018-06-24,33.7,0
7,1984-10-17,2018-06-21,2018-06-21,33.7,0
8,1984-10-17,2018-06-22,2018-06-22,33.7,0
9,1984-10-17,2018-07-31,2018-07-31,33.8,0




————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————


DataFrame baru 'df_gabungan_reordered_clean_clean' telah dibuat.
Jumlah kolom di DataFrame LAMA: 33
Jumlah kolom di DataFrame BARU: 21

Jumlah baris (baru): 36491
Jumlah kolom (baru): 21
Ukuran di memori (baru): 34.40 MB

Tipe data setiap kolom (DataFrame BARU):
no_peserta              int64
no_keluarga             int64
hub_keluarga           object
gender                 object
status_pernikahan      object
segmen_peserta         object
prov_peserta           object
kab_kota_peserta      

,no_peserta,no_keluarga,usia_pasien,hub_keluarga,gender,status_pernikahan,segmen_peserta,prov_peserta,kab_kota_peserta,bobot,durasi_rawat,kelas_rawat,prov_faskes,kab_kota_faskes,kepemilikan_faskes,jenis_faskes,tipe_faskes,tingkat_layanan,nama_diagnosis,nama_tindakan,biaya_tagihan
0,65638,81863275,34.2,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,0,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,"Tuberculosis of lung, confirmed by sputum micr...",Evakuasi medis / Ambulans Darat,480000
1,65638,81863275,34.3,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,0,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,Acute abdomen,Evakuasi medis / Ambulans Darat,480000
2,65638,81863275,33.3,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,0,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,DOKTER PRAKTER PERORANGAN,RJTP,Other retinal detachments,Evakuasi medis / Ambulans Darat,580000
3,65638,81863275,34.1,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,0,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,"Tuberculosis of lung, confirmed by sputum micr...",Evakuasi medis / Ambulans Darat,480000
4,65638,81863275,34.4,ISTRI,PEREMPUAN,KAWIN,PPU,RIAU,INDRAGIRI HULU,12.892989,0,KELAS II,RIAU,PELALAWAN,SWASTA,KLINIK PRATAMA,KLINIK RAWAT INAP,RJTP,Abdominal and pelvic pain,Evakuasi medis / Ambulans Darat,480000
5,66027,65799350,70.3,ISTRI,PEREMPUAN,KAWIN,PBI APBN,SULAWESI SELATAN,SINJAI,18.593020,7,KELAS III,SULAWESI SELATAN,SINJAI,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,RAWAT INAP,RITP,"Iron deficiency anaemia, unspecified",Evakuasi medis / Ambulans Darat,52500
6,68930,68930,33.7,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,0,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RJTP,Routine postpartum follow-up,Pelayanan PNC 2 (Dua),25000
7,68930,68930,33.7,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,0,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RITP,Other single spontaneous delivery,Paket Persalinan per Vaginam normal (oleh Bidan),700000
8,68930,68930,33.7,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,0,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RJTP,Routine postpartum follow-up,Pelayanan PNC 1 (Satu),25000
9,68930,68930,33.8,PESERTA,PEREMPUAN,KAWIN,PBI APBD,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,1.627409,0,KELAS III,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,NON RAWAT INAP,RJTP,Routine postpartum follow-up,Pelayanan PNC 4 (Empat),25000


### **2.4 Identifikasi Outlier dan Distribusi Data**
Identifikasi outlier dan distribusi data berfungsi untuk memahami pola sebaran nilai dan tingkat ekstremitas data numerik. Hasil ini membantu menentukan strategi pemodelan yang tepat. Misalnya, jika distribusi tidak normal dan terdapat banyak outlier, maka algoritma berbasis pohon seperti CatBoost, XGBoost, LightGBM, atau Random Forest lebih cocok karena tidak sensitif terhadap skala maupun distribusi data.

---

In [ ]:
# -----------------------------------------------------------------------------
# Identifikasi Outlier dan Distribusi Data
# -----------------------------------------------------------------------------

# List kolom numerik yang ingin dicek outlier
num_cols = ['usia_pasien', 'durasi_rawat', 'bobot']

print("="*50)
print("Deteksi Outlier")
print("="*50)

for col in num_cols:
    data = df_gabungan_reordered_clean_clean[col].dropna()

    # Hitung Q1, Q3, dan IQR
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1

    # Tentukan batas bawah dan atas
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Deteksi outlier
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    perc = len(outliers) / len(data) * 100

    print(f"\nKolom: {col}")
    print(f"Jumlah outlier: {len(outliers)} ({perc:.2f}%)")

    # Menampilkan nilai outlier (maksimal 10 nilai)
    if len(outliers) > 0:
        print("Contoh outlier:", outliers.head(10).values)

print("\n" + "="*50)
print("Uji Normalitas (Kolom Numerik)")
print("="*50)

for col in num_cols:
    stat, p = kstest(df_gabungan_reordered_clean_clean[col].dropna(), 'norm')
    print(f"\nKolom: {col}")
    print(f"Statistics = {stat:.3f}, p = {p:.3f}")
    if p > 0.05:
        print("Distribusi kemungkinan normal")
    else:
        print("Distribusi tidak normal")


Deteksi Outlier

Kolom: usia_pasien
Jumlah outlier: 1 (0.00%)
Contoh outlier: [104.5]

Kolom: durasi_rawat
Jumlah outlier: 3029 (8.30%)
Contoh outlier: [7 3 1 1 3 2 2 4 1 3]

Kolom: bobot
Jumlah outlier: 2943 (8.07%)
Contoh outlier: [56.721382 56.721382 93.232056 93.232056 93.232056 93.232056 93.232056
 93.232056 93.232056 93.232056]

Uji Normalitas (Kolom Numerik)

Kolom: usia_pasien
Statistics = 0.994, p = 0.000
Distribusi tidak normal

Kolom: durasi_rawat
Statistics = 0.500, p = 0.000
Distribusi tidak normal

Kolom: bobot
Statistics = 0.797, p = 0.000
Distribusi tidak normal


**Interpretasi Hasil Deteksi Outlier**

Dari hasil deteksi outlier pada tiga variabel numerik (usia_pasien, durasi_rawat, dan bobot) ditemukan bahwa secara umum keberadaan outlier masih dalam batas yang dapat diterima secara statistik maupun logis.

* Usia pasien hanya memiliki satu nilai ekstrem (104,5 tahun). Nilai ini masih mungkin terjadi secara biologis dan tidak perlu dihapus.
* Durasi rawat dan bobot memiliki sekitar 8% data yang terdeteksi sebagai outlier. Namun, kedua variabel ini berkaitan langsung dengan kondisi klinis dan karakteristik pasien, sehingga variasinya justru merepresentasikan realitas lapangan yang kompleks, bukan kesalahan input.

Oleh karena itu, tidak dilakukan proses capping maupun penghapusan outlier, karena:

* Variasi data adalah bagian dari realitas klinis yang ingin dipertahankan untuk menjaga representativitas model.
* Capping atau dropping dapat mengurangi keragaman alami data dan berpotensi menurunkan korelasi serta sensitivitas model terhadap kasus ekstrem yang relevan secara medis.
* Outlier yang ada tidak mengganggu distribusi global data secara signifikan, sehingga tidak ada indikasi distorsi besar terhadap parameter statistik utama.

Dengan demikian, outlier tetap dipertahankan sebagai bentuk pelestarian konteks dan keaslian distribusi data, sambil memastikan model tetap belajar dari seluruh spektrum kondisi pasien yang mungkin terjadi di dunia nyata.

---
**Interpretasi Uji Normalitas**

Hasil uji menunjukkan bahwa usia_pasien, durasi_rawat, dan bobot semuanya tidak berdistribusi normal (p < 0,05). Ini wajar karena data kesehatan umumnya bersifat skewed akibat variasi kondisi pasien, misalnya lama rawat ekstrem atau bobot yang sangat berbeda antar kasus.

Ketidaknormalan ini tidak menjadi masalah utama, sebab analisis selanjutnya dapat menggunakan pendekatan non-parametrik seperti korelasi Spearman yang tidak bergantung pada distribusi normal. Selain itu, model pembelajaran mesin modern pun relatif tahan terhadap data non-normal.

Maka, tidak perlu dilakukan transformasi paksa, karena justru bisa mengaburkan makna asli data. Dalam konteks klinis, ketidakteraturan adalah bagian alami dari realitas yang sedang dipelajari.

---

### **2.5 Analisis Korelasi Numerik dan Kategorikal terhadap Target "PNK17" (biaya_tagihan)**

Analisis korelasi untuk data numerik dilakukan menggunakan uji Spearman, sedangkan untuk data kategorial digunakan ANOVA atau Point Biserial. Kolom no_peserta dan no_keluarga dikecualikan karena tidak relevan. Di bawah ini ditampilkan hasil korelasi beserta catatan strategi, yang menunjukkan alasan mempertahankan seluruh 18 prediktor (15 kategorial dan 3 numerik) untuk memprediksi target biaya_tagihan (numerik).

---

1. **Korelasi Sangat Kuat:** Fitur-fitur ini memiliki F-statistik yang sangat tinggi, menunjukkan perbedaan yang sangat besar antar grup. Ini adalah prediktor terkuat Anda.

* tingkat_layanan (F-statistik: 11462,065)
* nama_tindakan (F-statistik: 2346,254)

---

2. **Korelasi Kuat:**
Fitur-fitur ini juga menunjukkan F-statistik yang sangat tinggi, menjadikannya prediktor yang sangat kuat.

* tipe_faskes (F-statistik: 767,864)
* jenis_faskes (F-statistik: 602,800)
* kelas_rawat (F-statistik: 535,596)
* status_pernikahan (F-statistik: 534,497)

---

3. **Korelasi Sedang:**
Kelompok ini memiliki hubungan yang jelas dan signifikan secara statistik. durasi_rawat adalah satu-satunya fitur numerik yang memiliki korelasi yang cukup baik (positif sedang).

* hub_keluarga (F-statistik: 351,393)
* segmen_peserta (F-statistik: 344,472)
* kepemilikan_faskes (F-statistik: 169,560)
* durasi_rawat (Korelasi Spearman: 0,406)

---
4. **Korelasi Lemah:**
Fitur-fitur ini masih memiliki hubungan yang signifikan (p-value = 0), tetapi kekuatan hubungannya (F-statistik) tidak sebesar kelompok di atas.

* prov_faskes (F-statistik: 80,948)
* prov_peserta (F-statistik: 80,493)
* nama_diagnosis (F-statistik: 47,309)
* kab_kota_faskes (F-statistik: 21,409)
* kab_kota_peserta (F-statistik: 21,166)

---

5. **Korelasi Sangat Lemah/Dapat Diabaikan:**
Fitur-fitur ini menunjukkan korelasi yang sangat kecil (mendekati 0). gender masih signifikan secara statistik (p-value kecil), tetapi kekuatan korelasinya sangat rendah.

* usia_pasien (Korelasi Spearman: -0,145)
* bobot (Korelasi Spearman: -0,128)
* gender (Korelasi Biserial: -0,028)

---
**Catatan Penting: Strategi Uji Kombinasi Fitur**

Saya tidak akan mengeksklusi (membuang) satu pun dari 18 prediktor ini, meskipun ada yang masuk kategori "Lemah" atau "Sangat Lemah" (seperti gender atau usia_pasien).

Analisis korelasi ini bersifat univariate (satu-lawan-satu). Analisis ini tidak dapat melihat kekuatan interaksi antar fitur. Sebuah fitur yang terlihat lemah sendirian mungkin menjadi sangat kuat jika dikombinasikan dengan fitur lain.

Oleh karena itu, semua 18 fitur ini akan tetap digunakan dan akan diuji kekuatannya secara multivariate (bersama-sama) sebagai kombinasi pada tahap feature importance modelling.

---

In [ ]:
# -----------------------------------------------------------------------------
# Analisis Korelasi Numerik dan Kategorikal terhadap Target biaya_tagihan
# -----------------------------------------------------------------------------

# Korelasi Numerik (Spearman)
print("="*50)
print("Hasil Uji Korelasi Numerik (Spearman)")
print("="*50)

num_cols = ['usia_pasien', 'bobot', 'durasi_rawat']
target = 'biaya_tagihan'

for col in num_cols:
    corr = df_gabungan_reordered_clean_clean[col].corr(df_gabungan_reordered_clean_clean[target], method='spearman')
    print(f"Kolom: {col}")
    print(f"Korelasi dengan {target}: {corr:.3f}\n")

# Korelasi Kategorikal (ANOVA / Point Biserial)
print("="*50)
print("Hasil Uji Korelasi Kategorikal (ANOVA / Point Biserial)")
print("="*50)

cat_cols = ['hub_keluarga', 'gender', 'status_pernikahan', 'segmen_peserta',
            'prov_peserta', 'kab_kota_peserta', 'kelas_rawat', 'prov_faskes',
            'kab_kota_faskes', 'kepemilikan_faskes', 'jenis_faskes', 'tipe_faskes',
            'tingkat_layanan', 'nama_diagnosis', 'nama_tindakan']

for col in cat_cols:
    kategori = df_gabungan_reordered_clean_clean[col].dropna().unique()
    print(f"\nKolom: {col} → Jumlah kategori: {len(kategori)}")

    if len(kategori) == 2:
        # Binary → Point Biserial
        mapping = {k:i for i,k in enumerate(kategori)}
        col_num = df_gabungan_reordered_clean_clean[col].map(mapping)
        corr, pval = pointbiserialr(col_num, df_gabungan_reordered_clean_clean[target])
        print(f"Binary → Korelasi: {corr:.3f}, p-value: {pval:.3e}")
    else:
        # >2 kategori → ANOVA
        grup = [df_gabungan_reordered_clean_clean[df_gabungan_reordered_clean_clean[col]==k][target] for k in kategori]
        f_stat, p_val = f_oneway(*grup)
        print(f"Multi → F-statistik: {f_stat:.3f}, p-value: {p_val:.3e}")

print("\n" + "="*50 + "\n")


Hasil Uji Korelasi Numerik (Spearman)
Kolom: usia_pasien
Korelasi dengan biaya_tagihan: -0.145

Kolom: bobot
Korelasi dengan biaya_tagihan: -0.128

Kolom: durasi_rawat
Korelasi dengan biaya_tagihan: 0.406

Hasil Uji Korelasi Kategorikal (ANOVA / Point Biserial)

Kolom: hub_keluarga → Jumlah kategori: 5
Multi → F-statistik: 351.393, p-value: 2.225e-297

Kolom: gender → Jumlah kategori: 2
Binary → Korelasi: -0.028, p-value: 1.313e-07

Kolom: status_pernikahan → Jumlah kategori: 3
Multi → F-statistik: 534.497, p-value: 1.608e-229

Kolom: segmen_peserta → Jumlah kategori: 5
Multi → F-statistik: 344.472, p-value: 1.348e-291

Kolom: prov_peserta → Jumlah kategori: 34
Multi → F-statistik: 80.493, p-value: 0.000e+00

Kolom: kab_kota_peserta → Jumlah kategori: 485
Multi → F-statistik: 21.166, p-value: 0.000e+00

Kolom: kelas_rawat → Jumlah kategori: 3
Multi → F-statistik: 535.596, p-value: 5.526e-230

Kolom: prov_faskes → Jumlah kategori: 34
Multi → F-statistik: 80.948, p-value: 0.000e+00

Kolo

## **3. Train/Test Data — pra-modelling (preparation)**
Train/test split adalah langkah krusial untuk menilai kemampuan model dalam menggeneralisasi pola, bukan sekadar menghafal data. Rasio 80:20 digunakan karena memberi keseimbangan antara kecukupan data untuk belajar dan keandalan evaluasi performa. 80% data memungkinkan model menangkap variasi dan struktur dengan stabil, sementara 20% sisanya menjadi cermin objektif untuk menguji kejujuran prediksinya di dunia nyata. Rasio ini bukan angka mutlak, melainkan kompromi rasional agar hasil pelatihan tetap representatif tanpa mengorbankan validitas evaluasi.

---

In [ ]:
warnings.filterwarnings('ignore')

# --- Definisikan target dan fitur ---
target = 'biaya_tagihan'

categorical_features = [
    'hub_keluarga','status_pernikahan', 'segmen_peserta', 'kelas_rawat', 'tingkat_layanan',
    'nama_diagnosis', 'nama_tindakan', 'tipe_faskes', 'jenis_faskes', 'kepemilikan_faskes', 'kab_kota_faskes',
    'prov_faskes', 'kab_kota_peserta', 'prov_peserta', 'gender'
]
numeric_features = ['usia_pasien', 'durasi_rawat', 'bobot']
all_features = categorical_features + numeric_features

X = df_gabungan_reordered_clean_clean[all_features]
y = df_gabungan_reordered_clean_clean[target]

# --- Train/Test Split ---
# test_size=0.2 artinya 20% untuk data tes, 80% untuk data latih
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nData berhasil dibagi:")
print(f"Data Latih (Train) : {len(X_train)} baris (80.0%)")
print(f"Data Tes (Test)    : {len(X_test)} baris (20.0%)")


Data berhasil dibagi:
Data Latih (Train) : 29192 baris (80.0%)
Data Tes (Test)    : 7299 baris (20.0%)


## **4. Data Modelling**
Pada data tabel yang telah siap untuk tahap modelling, terdapat tiga hal krusial yang perlu diperhatikan. Pertama, terdapat 15 fitur kategorikal dan 3 fitur numerik sebagai prediktor. Kedua, data mengandung outlier. Ketiga, distribusi data numerik tidak bersifat normal. Target yang akan diprediksi adalah biaya_tagihan dengan tipe numerik.

*Oleh karena itu, algoritma yang dipilih harus mampu menangani ketiga karakteristik tersebut secara native. Jika tidak, maka perlu dibuat pipeline khusus agar algoritma yang digunakan tetap dapat memproses data dengan benar.*

---
Berdasarkan pertimbangan tersebut, diperoleh tiga kandidat algoritma yang mampu mengolah data kategorikal secara native, serta memiliki ketahanan terhadap keberadaan outlier dan distribusi data yang tidak normal:
1. CatBoost Regressor
2. LightGBM (LGBM) Regressor
3. HistGradientBoostingRegressor (dari Scikit-learn)

---
Terdapat pula tiga kandidat algoritma yang dapat digunakan, namun memerlukan pipeline terlebih dahulu karena belum mampu menangani data kategorikal, outlier, serta distribusi data yang tidak normal secara langsung. Oleh sebab itu, pipeline diterapkan agar data dapat diproses dengan optimal. Berikut ini algoritma yang saya pilih:
1. MLPRegressor
2. Ridge
3. RandomForest.

---





### **4.1 Perbandingan Algoritma yang Dapat Menangani Data Kategorikal, Outlier, dan Distribusi Tidak Normal secara Native**

Di sini kita akan membandingkan algoritma CatBoost, LightGBM (LGBM) Regressor, dan HistGradientBoostingRegressor secara adil, dengan menilai kinerja masing-masing melalui metrik MAE, RMSE, dan R² untuk menentukan algoritma yang paling unggul.

***Catatan: Tahap ini akan memakan waktu yang cukup lama, sehingga disarankan untuk meninggalkannya berjalan sendiri...***

---

In [ ]:
%%time
# --- Model Awal (Hanya untuk mendapat ranking fitur) ---
print("Melatih model awal (CatBoost) untuk mendapatkan feature importance...")

# Tentukan parameter model CatBoost (Hanya untuk ranking)
cb_model_params = {
    'iterations': 1000, 'learning_rate': 0.05, 'depth': 6,
    'eval_metric': 'RMSE', 'random_seed': 42,
    'early_stopping_rounds': 50, 'verbose': 0
}

# Asumsi X_train, y_train, categorical_features, all_features sudah ada
# (Dijalankan dari Sel 1)
train_pool_full = Pool(X_train, y_train, cat_features=categorical_features)
test_pool_full = Pool(X_test, y_test, cat_features=categorical_features)

model_full = CatBoostRegressor(**cb_model_params)
model_full.fit(train_pool_full, eval_set=test_pool_full)

# Dapatkan Feature Importance (FI)
fi_df = pd.DataFrame({
    'Fitur': all_features,
    'Importance': model_full.get_feature_importance()
}).sort_values(by='Importance', ascending=False)

print("\n=== Feature Importance (Ranking Teratas) ===")
display(fi_df)


# --- Definisikan Model Perbandingan (Fitur) ---
features_by_importance = fi_df['Fitur'].tolist()

multi_models = {
    "Model 1 (Top 3 Fitur)": features_by_importance[:3],
    "Model 2 (Top 5 Fitur)": features_by_importance[:5],
    "Model 3 (Top 10 Fitur)": features_by_importance[:10],
    "Model 4 (Top 15 Fitur)": features_by_importance[:15],
    "Model 5 (Semua 18 Fitur)": all_features
}

# --- Definisikan Model Perbandingan (Algoritma) ---
models_to_test = {
    "CatBoost": CatBoostRegressor(
        iterations=3000, learning_rate=0.03, depth=6,
        l2_leaf_reg=10, subsample=0.8, bootstrap_type='Bernoulli',
        eval_metric='RMSE', random_seed=42,
        early_stopping_rounds=100, verbose=0
    ),
    "LGBM": lgbm.LGBMRegressor(
        n_estimators=3000, learning_rate=0.03, num_leaves=2**6, # num_leaves ~ 2^depth
        reg_lambda=10, subsample=0.8,
        random_state=42, n_jobs=-1, verbose=-1 # verbose=-1 untuk silent
    ),
    "HistGBM": HistGradientBoostingRegressor(
        max_iter=3000, learning_rate=0.03, max_depth=6,
        l2_regularization=10,
        random_state=42, early_stopping=True,
        n_iter_no_change=100, verbose=0
    )
}

# Siapkan list untuk menampung hasil
results = []
print("\nMemulai pelatihan model perbandingan (Native Handlers Loop)...")

# =====================================================================
# --- Loop Pelatihan Model Perbandingan (CatBoost vs LGBM vs HistGBM) ---
# =====================================================================
# Loop Terluar: Iterasi berdasarkan JUMLAH FITUR
for model_name, feature_list in multi_models.items():

    print(f"\n--- Menguji Set Fitur: {model_name} ---")

    # 1. Tentukan fitur kategorial untuk iterasi INI
    current_cat_features = [col for col in feature_list if col in categorical_features]

    # 2. Buat data subset (Hanya kolom yang dibutuhkan)
    X_train_subset = X_train[feature_list]
    X_test_subset = X_test[feature_list]

    # 3. Loop Terdalam: Iterasi berdasarkan ALGORITMA
    for algo_name, model in models_to_test.items():

        print(f"  > Menguji Algoritma: {algo_name}...")

        try:
            # 4. Latih model sesuai cara "native" terbaiknya

            if algo_name == "CatBoost":
                train_pool = Pool(X_train_subset, y_train, cat_features=current_cat_features)
                test_pool = Pool(X_test_subset, y_test, cat_features=current_cat_features)

                cb_model = model.copy()
                cb_model.fit(train_pool, eval_set=test_pool)

                y_pred = cb_model.predict(X_test_subset)

            elif algo_name == "LGBM":
                # LGBM cara terbaiknya adalah mengubah dtype
                X_train_copy = X_train_subset.copy()
                X_test_copy = X_test_subset.copy()

                for col in current_cat_features:
                    X_train_copy[col] = X_train_copy[col].astype('category')
                    X_test_copy[col] = X_test_copy[col].astype('category')

                mod = model.__class__(**model.get_params())

                mod.fit(X_train_copy, y_train,
                        eval_set=[(X_test_copy, y_test)],
                        eval_metric='rmse',
                        callbacks=[lgbm.early_stopping(100, verbose=False)])

                y_pred = mod.predict(X_test_copy)

            elif algo_name == "HistGBM":
                # HistGBM mengharuskan data kategorial di-encode DULU jika > 255 kategori
                # Karena kita menguji 'native', kita akan biarkan dia error dan menangkapnya

                # Cek batasan kardinalitas
                exceeds_limit = False
                for col in current_cat_features:
                    if X_train_subset[col].nunique() > 255:
                        print(f"    ! Gagal (HistGBM): Fitur '{col}' memiliki > 255 kategori unik.")
                        exceeds_limit = True
                        break

                if exceeds_limit:
                    raise ValueError("HistGBM tidak mendukung > 255 kategori unik secara native.")

                # Jika tidak ada error, lanjutkan
                mod = model.__class__(**model.get_params())
                mod.set_params(categorical_features=current_cat_features) # Beritahu model mana kolom kategori

                mod.fit(X_train_subset, y_train)
                y_pred = mod.predict(X_test_subset)


            # --- Evaluasi ---
            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            rmse = math.sqrt(mse)
            r2 = r2_score(y_test, y_pred)

            # Simpan hasil
            results.append({
                "Model": f"{algo_name} - {model_name}", # Nama gabungan
                "Algoritma": algo_name,
                "Jumlah_Fitur": len(feature_list),
                "MAE": mae, "MSE": mse, "RMSE": rmse, "R2": r2
            })

        except Exception as e:
            # Jika error (mis. data terlalu sedikit atau error HistGBM)
            print(f"    ! Gagal melatih {algo_name}: {e}")
            results.append({
                "Model": f"{algo_name} - {model_name}",
                "Algoritma": algo_name,
                "Jumlah_Fitur": len(feature_list),
                "MAE": np.nan, "MSE": np.nan, "RMSE": np.nan, "R2": np.nan
            })

# --- 6. Tampilkan Tabel Hasil Perbandingan ---
print("\n=== HASIL PERBANDINGAN MODEL (NATIVE HANDLERS) ===")
df_results = pd.DataFrame(results)

# Urutkan berdasarkan R2 terbaik
df_results = df_results.sort_values(by="R2", ascending=False)

# Atur kolom yang ingin ditampilkan
display_cols = ['Model', 'Algoritma', 'Jumlah_Fitur', 'MAE', 'MSE', 'RMSE', 'R2']
display(df_results[display_cols])

Melatih model awal (CatBoost) untuk mendapatkan feature importance...

=== Feature Importance (Ranking Teratas) ===


,Fitur,Importance
4,tingkat_layanan,44.992298
6,nama_tindakan,25.771135
16,durasi_rawat,7.483587
11,prov_faskes,3.837953
14,gender,3.509527
13,prov_peserta,3.218013
7,tipe_faskes,2.099065
17,bobot,1.692532
5,nama_diagnosis,1.421860
10,kab_kota_faskes,1.235093



Memulai pelatihan model perbandingan (Native Handlers Loop)...

--- Menguji Set Fitur: Model 1 (Top 3 Fitur) ---
  > Menguji Algoritma: CatBoost...
  > Menguji Algoritma: LGBM...
  > Menguji Algoritma: HistGBM...

--- Menguji Set Fitur: Model 2 (Top 5 Fitur) ---
  > Menguji Algoritma: CatBoost...
  > Menguji Algoritma: LGBM...
  > Menguji Algoritma: HistGBM...

--- Menguji Set Fitur: Model 3 (Top 10 Fitur) ---
  > Menguji Algoritma: CatBoost...
  > Menguji Algoritma: LGBM...
  > Menguji Algoritma: HistGBM...
    ! Gagal (HistGBM): Fitur 'nama_diagnosis' memiliki > 255 kategori unik.
    ! Gagal melatih HistGBM: HistGBM tidak mendukung > 255 kategori unik secara native.

--- Menguji Set Fitur: Model 4 (Top 15 Fitur) ---
  > Menguji Algoritma: CatBoost...
  > Menguji Algoritma: LGBM...
  > Menguji Algoritma: HistGBM...
    ! Gagal (HistGBM): Fitur 'nama_diagnosis' memiliki > 255 kategori unik.
    ! Gagal melatih HistGBM: HistGBM tidak mendukung > 255 kategori unik secara native.

--- M

,Model,Algoritma,Jumlah_Fitur,MAE,MSE,RMSE,R2
7,LGBM - Model 3 (Top 10 Fitur),LGBM,10,21351.975099,7.101288e+09,84269.140104,0.850737
13,LGBM - Model 5 (Semua 18 Fitur),LGBM,18,21331.603637,8.029954e+09,89610.011203,0.831217
10,LGBM - Model 4 (Top 15 Fitur),LGBM,15,21445.958718,8.176163e+09,90422.137832,0.828144
9,CatBoost - Model 4 (Top 15 Fitur),CatBoost,15,28952.059805,9.774678e+09,98866.971059,0.794545
6,CatBoost - Model 3 (Top 10 Fitur),CatBoost,10,29125.133189,9.823507e+09,99113.607443,0.793518
12,CatBoost - Model 5 (Semua 18 Fitur),CatBoost,18,29326.919533,1.007786e+10,100388.555122,0.788172
4,LGBM - Model 2 (Top 5 Fitur),LGBM,5,27471.531944,1.030404e+10,101508.838145,0.783418
5,HistGBM - Model 2 (Top 5 Fitur),HistGBM,5,25086.652614,1.031375e+10,101556.627104,0.783214
1,LGBM - Model 1 (Top 3 Fitur),LGBM,3,28286.675733,1.124974e+10,106064.804168,0.763540
2,HistGBM - Model 1 (Top 3 Fitur),HistGBM,3,28643.318556,1.151022e+10,107285.708600,0.758065


CPU times: user 18min 51s, sys: 1min 2s, total: 19min 54s
Wall time: 13min 6s


Jadi, dari hasil perbandingan di atas, model terbaik adalah Model 3 (Top 10 fitur) yang menggunakan algoritma LGBM, dengan kinerja MAE = 21.351,98, RMSE = 84.269,14, dan R² = 0,851. Artinya, secara rata-rata model menyimpang sekitar 21.352 unit dari biaya tagihan aktual (MAE), sementara RMSE yang lebih besar menekankan adanya beberapa prediksi yang jauh meleset, sehingga kedua metrik ini sebenarnya menggambarkan besaran kesalahan prediksi dalam satuan uang. R² yang tinggi menunjukkan bahwa model masih mampu menjelaskan sekitar 85% variasi biaya tagihan, jadi prediksi umumnya cukup akurat, tetapi prediksi ekstrem perlu diperhatikan.

---

### **4.2 Perbandingan Algoritma yang Memerlukan Pipeline untuk Menangani Data Kategorikal, Outlier, dan Distribusi Tidak Normal**

Di sini kita akan membandingkan algoritma MLPRegressor, Ridge, dan RandomForest secara adil, dengan terlebih dahulu menerapkan pipeline untuk menangani data kategorikal, outlier, dan distribusi yang tidak normal. Kinerja masing-masing algoritma akan dinilai menggunakan metrik MAE, RMSE, dan R² untuk menentukan algoritma yang paling unggul.

***Catatan: Tahap ini akan memakan waktu yang lama, sehingga disarankan untuk meninggalkannya berjalan sendiri...***

---

In [ ]:
%%time
# --- Model Awal (Hanya untuk mendapat ranking fitur) ---
# (Kita tetap pakai CatBoost di sini HANYA untuk ranking, karena cepat & mudah)
print("Melatih model awal (CatBoost) untuk mendapatkan feature importance...")

# Tentukan parameter model CatBoost (Hanya untuk ranking)
cb_model_params = {
    'iterations': 1000, 'learning_rate': 0.05, 'depth': 6,
    'eval_metric': 'RMSE', 'random_seed': 42,
    'early_stopping_rounds': 50, 'verbose': 0
}
# (Asumsi X_train, y_train, categorical_features, all_features sudah ada dari sel sebelumnya)
train_pool_full = Pool(X_train, y_train, cat_features=categorical_features)
test_pool_full = Pool(X_test, y_test, cat_features=categorical_features)
model_full = CatBoostRegressor(**cb_model_params)
model_full.fit(train_pool_full, eval_set=test_pool_full)

# Dapatkan Feature Importance (FI)
fi_df = pd.DataFrame({
    'Fitur': all_features,
    'Importance': model_full.get_feature_importance()
}).sort_values(by='Importance', ascending=False)

print("\n=== Feature Importance (Ranking Teratas) ===")
display(fi_df)


# --- Definisikan Model Perbandingan (Fitur) ---
features_by_importance = fi_df['Fitur'].tolist()
multi_models = {
    "Model 1 (Top 3 Fitur)": features_by_importance[:3],
    "Model 2 (Top 5 Fitur)": features_by_importance[:5],
    "Model 3 (Top 10 Fitur)": features_by_importance[:10],
    "Model 4 (Top 15 Fitur)": features_by_importance[:15],
    "Model 5 (Semua 18 Fitur)": all_features
}

# --- Definisikan Model Perbandingan (Algoritma) ---
# Ini adalah model-model yang akan kita adu
models_to_test = {
    "MLP": MLPRegressor(
        hidden_layer_sizes=(100, 50), max_iter=500, # Kurangi iterasi agar tidak terlalu lama
        random_state=42, early_stopping=True, # Tetap pakai early stopping
        learning_rate_init=0.001, alpha=0.01
    ),
    "RandomForest": RandomForestRegressor(
        n_estimators=100, # 100 pohon sudah cukup baik
        max_depth=10,     # Mencegah overfitting
        random_state=42,
        n_jobs=-1         # Gunakan semua core CPU agar cepat
    ),
    "Ridge": Ridge(
        alpha=1.0, # Regularisasi standar
        random_state=42
    )
}

# Siapkan list untuk menampung hasil
results1 = []
print("\nMemulai pelatihan model perbandingan (Championship Loop)...")

# =====================================================================
# --- Loop Pelatihan Model Perbandingan (MLP vs RF vs Ridge) ---
# =====================================================================
# Loop Terluar: Iterasi berdasarkan JUMLAH FITUR
for model_name, feature_list in multi_models.items():

    print(f"\n--- Menguji Set Fitur: {model_name} ---")

    # 1. Tentukan fitur numerik dan kategorial untuk iterasi INI
    current_cat_features = [col for col in feature_list if col in categorical_features]
    current_num_features = [col for col in feature_list if col in numeric_features]

    # 2. Buat pipeline preprocessing (SAMA UNTUK SEMUA MODEL)
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, current_num_features),
            ('cat', categorical_transformer, current_cat_features)
        ],
        remainder='passthrough'
    )

    # 3. Buat data subset (Hanya kolom yang dibutuhkan)
    X_train_subset = X_train[feature_list]
    X_test_subset = X_test[feature_list]

    # 4. Loop Terdalam: Iterasi berdasarkan ALGORITMA
    for algo_name, model in models_to_test.items():

        print(f"  > Menguji Algoritma: {algo_name}...")

        # 5. Buat pipeline model LENGKAP (Preprocessing + Model)
        full_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])

        # 6. Latih model
        try:
            full_pipeline.fit(X_train_subset, y_train)

            # --- Evaluasi ---
            y_pred = full_pipeline.predict(X_test_subset)

            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            rmse = math.sqrt(mse)
            r2 = r2_score(y_test, y_pred)

            # Simpan hasil
            results1.append({
                "Model": f"{algo_name} - {model_name}", # Nama gabungan
                "Algoritma": algo_name,
                "Jumlah_Fitur": len(feature_list),
                "MAE": mae, "MSE": mse, "RMSE": rmse, "R2": r2
            })

        except Exception as e:
            # Jika error (mis. data terlalu sedikit untuk 1 fitur)
            print(f"    ! Gagal melatih {algo_name}: {e}")
            results1.append({
                "Model": f"{algo_name} - {model_name}",
                "Algoritma": algo_name,
                "Jumlah_Fitur": len(feature_list),
                "MAE": np.nan, "MSE": np.nan, "RMSE": np.nan, "R2": np.nan
            })

# --- 6. Tampilkan Tabel Hasil Perbandingan ---
print("\n=== HASIL PERBANDINGAN MODEL (MLP vs RF vs Ridge) ===")
df_results1 = pd.DataFrame(results1)

# Urutkan berdasarkan R2 terbaik
df_results1 = df_results1.sort_values(by="R2", ascending=False)

# Atur kolom yang ingin ditampilkan
display_cols = ['Model', 'Algoritma', 'Jumlah_Fitur', 'MAE', 'MSE', 'RMSE', 'R2']
display(df_results1[display_cols])

Melatih model awal (CatBoost) untuk mendapatkan feature importance...

=== Feature Importance (Ranking Teratas) ===


,Fitur,Importance
4,tingkat_layanan,44.992298
6,nama_tindakan,25.771135
16,durasi_rawat,7.483587
11,prov_faskes,3.837953
14,gender,3.509527
13,prov_peserta,3.218013
7,tipe_faskes,2.099065
17,bobot,1.692532
5,nama_diagnosis,1.421860
10,kab_kota_faskes,1.235093



Memulai pelatihan model perbandingan (Championship Loop)...

--- Menguji Set Fitur: Model 1 (Top 3 Fitur) ---
  > Menguji Algoritma: MLP...
  > Menguji Algoritma: RandomForest...
  > Menguji Algoritma: Ridge...

--- Menguji Set Fitur: Model 2 (Top 5 Fitur) ---
  > Menguji Algoritma: MLP...
  > Menguji Algoritma: RandomForest...
  > Menguji Algoritma: Ridge...

--- Menguji Set Fitur: Model 3 (Top 10 Fitur) ---
  > Menguji Algoritma: MLP...
  > Menguji Algoritma: RandomForest...
  > Menguji Algoritma: Ridge...

--- Menguji Set Fitur: Model 4 (Top 15 Fitur) ---
  > Menguji Algoritma: MLP...
  > Menguji Algoritma: RandomForest...
  > Menguji Algoritma: Ridge...

--- Menguji Set Fitur: Model 5 (Semua 18 Fitur) ---
  > Menguji Algoritma: MLP...
  > Menguji Algoritma: RandomForest...
  > Menguji Algoritma: Ridge...

=== HASIL PERBANDINGAN MODEL (MLP vs RF vs Ridge) ===


,Model,Algoritma,Jumlah_Fitur,MAE,MSE,RMSE,R2
10,RandomForest - Model 4 (Top 15 Fitur),RandomForest,15,27839.037311,6.635260e+09,81457.105557,0.860533
9,MLP - Model 4 (Top 15 Fitur),MLP,15,25214.377186,6.689843e+09,81791.463185,0.859385
13,RandomForest - Model 5 (Semua 18 Fitur),RandomForest,18,27949.752383,6.790430e+09,82404.065220,0.857271
12,MLP - Model 5 (Semua 18 Fitur),MLP,18,25140.290250,7.044470e+09,83931.338822,0.851931
7,RandomForest - Model 3 (Top 10 Fitur),RandomForest,10,28161.993319,7.168493e+09,84666.952943,0.849324
11,Ridge - Model 4 (Top 15 Fitur),Ridge,15,36990.669332,8.493609e+09,92160.780578,0.821472
14,Ridge - Model 5 (Semua 18 Fitur),Ridge,18,37099.938721,8.511666e+09,92258.689717,0.821092
8,Ridge - Model 3 (Top 10 Fitur),Ridge,10,36902.935477,8.572166e+09,92585.992174,0.819820
6,MLP - Model 3 (Top 10 Fitur),MLP,10,33050.502274,1.066472e+10,103270.143648,0.775837
1,RandomForest - Model 1 (Top 3 Fitur),RandomForest,3,32469.572747,1.133005e+10,106442.726100,0.761852


CPU times: user 52min 44s, sys: 7.76 s, total: 52min 52s
Wall time: 31min 40s


Jadi, dari hasil perbandingan di atas, model terbaik adalah Model 4 (Top 15 fitur) yang menggunakan algoritma RandomForest, dengan kinerja MAE = 27.839,04, RMSE = 81.457,11, dan R² = 0,861. Artinya, secara rata-rata model menyimpang sekitar 27.839 unit dari biaya tagihan aktual (MAE), sementara RMSE yang lebih besar menunjukkan adanya beberapa prediksi yang jauh meleset, sehingga kedua metrik ini menggambarkan besaran kesalahan prediksi dalam satuan uang. Nilai R² yang tinggi menunjukkan model mampu menjelaskan sekitar 86% variasi biaya tagihan, sehingga prediksi umumnya cukup akurat, meskipun prediksi untuk kasus ekstrem tetap perlu diperhatikan.

---

### **4.3 Pemilihan Algoritma Final dan Pelatihan Ulang Model**

Dari hasil perbandingan, algoritma yang dapat menangani data kategorikal, outlier, dan distribusi tidak normal secara native dimenangkan oleh Model 3 (Top 10 fitur) dengan algoritma LGBM, memiliki kinerja MAE = 21.351,98, RMSE = 84.269,14, dan R² = 0,851.

Sementara itu, algoritma yang memerlukan pipeline untuk menangani data serupa dimenangkan oleh Model 4 (Top 15 fitur) dengan algoritma RandomForest, memiliki kinerja MAE = 27.839,04, RMSE = 81.457,11, dan R² = 0,861.

Maka pilihan algoritma tergantung pada prioritas:

* Jika mengutamakan efisiensi waktu, kesalahan rata-rata yang lebih kecil dan kemudahan tanpa pipeline, LGBM (Model 3) lebih sesuai.

* Jika mengutamakan kemampuan menjelaskan variasi data yang sedikit lebih tinggi (R²) meskipun MAE lebih besar dan perlu pipeline, maka RandomForest (Model 4) bisa dipilih.

Secara praktis, untuk prediksi biaya tagihan yang konsisten dan implementasi lebih sederhana, **saya memilih  LGBM — Model 3 (Top 10 fitur) menjadi pilihan utama.**

---
***Pada Tahap ini, kita akan melakukan pelatihan ulang terhadap 5 model menggunakan algoritma LGBM untuk menampilkan daftar prediktor, waktu eksekusi, serta metrik kinerja masing-masing model, yaitu MAE, RMSE, dan R².***

---

In [ ]:
import time

# -----------------------------------------------------------------
# --- Model Awal (Hanya untuk mendapat ranking fitur) ---
# -----------------------------------------------------------------
print("Melatih model awal (CatBoost) untuk mendapatkan feature importance...")

cb_model_params = {
    'iterations': 1000, 'learning_rate': 0.05, 'depth': 6,
    'eval_metric': 'RMSE', 'random_seed': 42,
    'early_stopping_rounds': 50, 'verbose': 0
}

train_pool_full = Pool(X_train, y_train, cat_features=categorical_features)
test_pool_full = Pool(X_test, y_test, cat_features=categorical_features)

model_full = CatBoostRegressor(**cb_model_params)
model_full.fit(train_pool_full, eval_set=test_pool_full)

fi_df = pd.DataFrame({
    'Fitur': all_features,
    'Importance': model_full.get_feature_importance()
}).sort_values(by='Importance', ascending=False)

print("\n=== Feature Importance (Ranking Teratas) ===")
display(fi_df.head(15))


# -----------------------------------------------------------------
# --- Definisikan Model Perbandingan (Fitur) ---
# -----------------------------------------------------------------
features_by_importance = fi_df['Fitur'].tolist()

multi_models = {
    "Model 1 (Top 3 Fitur)": features_by_importance[:3],
    "Model 2 (Top 5 Fitur)": features_by_importance[:5],
    "Model 3 (Top 10 Fitur)": features_by_importance[:10],
    "Model 4 (Top 15 Fitur)": features_by_importance[:15],
    "Model 5 (Semua 18 Fitur)": all_features
}

# -----------------------------------------------------------------
# --- Definisikan Model Perbandingan (HANYA LGBM) ---
# -----------------------------------------------------------------
models_to_test = {
    "LGBM": lgbm.LGBMRegressor(
        n_estimators=3000, learning_rate=0.03, num_leaves=2**6,
        reg_lambda=10, subsample=0.8,
        random_state=42, n_jobs=-1, verbose=-1
    )
}

# --- Siapkan list untuk menampung hasil ---
lgbm_feature_results_list = []
print("\nMemulai pelatihan model perbandingan (Hanya LGBM)...")

# =====================================================================
# --- Loop Pelatihan Model Perbandingan (Hanya LGBM) ---
# =====================================================================
for model_name, feature_list in multi_models.items():

    print(f"\n--- Menguji Set Fitur: {model_name} ---")

    # =====================================================================
    # --- TAMBAHAN: Cetak daftar fitur yang digunakan ---
    # =====================================================================
    print(f"    Kombinasi Prediktor: {feature_list}")
    # =====================================================================

    current_cat_features = [col for col in feature_list if col in categorical_features]
    X_train_subset = X_train[feature_list]
    X_test_subset = X_test[feature_list]

    for algo_name, model in models_to_test.items():

        print(f"  > Menguji Algoritma: {algo_name}...")

        try:
            X_train_copy = X_train_subset.copy()
            X_test_copy = X_test_subset.copy()

            for col in current_cat_features:
                X_train_copy[col] = X_train_copy[col].astype('category')
                X_test_copy[col] = X_test_copy[col].astype('category')

            mod = model.__class__(**model.get_params())

            start_time = time.time()

            mod.fit(X_train_copy, y_train,
                    eval_set=[(X_test_copy, y_test)],
                    eval_metric='rmse',
                    callbacks=[lgbm.early_stopping(100, verbose=False)])

            end_time = time.time()
            training_time = end_time - start_time

            y_pred = mod.predict(X_test_copy)

            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            rmse = math.sqrt(mse)
            r2 = r2_score(y_test, y_pred)

            lgbm_feature_results_list.append({
                "Model": f"{algo_name} - {model_name}",
                "Algoritma": algo_name,
                "Jumlah_Fitur": len(feature_list),
                "Waktu_Latih_Detik": training_time,
                "MAE": mae,
                "MSE": mse,
                "RMSE": rmse,
                "R2": r2,
                "Fitur_yang_Digunakan": feature_list
            })

            print(f"    > Selesai. Waktu Latih: {training_time:.2f} detik. R2: {r2:.5f}")

        except Exception as e:
            print(f"    ! Gagal melatih {algo_name}: {e}")
            lgbm_feature_results_list.append({
                "Model": f"{algo_name} - {model_name}",
                "Algoritma": algo_name,
                "Jumlah_Fitur": len(feature_list),
                "Waktu_Latih_Detik": np.nan,
                "MAE": np.nan, "MSE": np.nan, "RMSE": np.nan, "R2": np.nan,
                "Fitur_yang_Digunakan": feature_list
            })

# =====================================================================
# --- Tampilkan Tabel Hasil Perbandingan ---
# =====================================================================
print("\n=== HASIL PERBANDINGAN MODEL (HANYA LGBM) ===")
df_lgbm_feature_comparison = pd.DataFrame(lgbm_feature_results_list)

df_lgbm_feature_comparison = df_lgbm_feature_comparison.sort_values(by="R2", ascending=False)

# Ganti nama kolom 'Fitur_yang_Digunakan' menjadi 'Kombinasi Prediktor'
df_lgbm_feature_comparison = df_lgbm_feature_comparison.rename(
    columns={'Fitur_yang_Digunakan': 'Kombinasi Prediktor'}
)

# Atur kolom yang ingin ditampilkan sesuai urutan yang diminta
display_cols = [
    'Model',
    'Kombinasi Prediktor',
    'MAE',
    'RMSE',
    'R2'
]

# Tampilkan DataFrame dengan kolom yang sudah dipilih dan diubah namanya
display(df_lgbm_feature_comparison[display_cols])

Melatih model awal (CatBoost) untuk mendapatkan feature importance...

=== Feature Importance (Ranking Teratas) ===


,Fitur,Importance
4,tingkat_layanan,44.992298
6,nama_tindakan,25.771135
16,durasi_rawat,7.483587
11,prov_faskes,3.837953
14,gender,3.509527
13,prov_peserta,3.218013
7,tipe_faskes,2.099065
17,bobot,1.692532
5,nama_diagnosis,1.421860
10,kab_kota_faskes,1.235093



Memulai pelatihan model perbandingan (Hanya LGBM)...

--- Menguji Set Fitur: Model 1 (Top 3 Fitur) ---
    Kombinasi Prediktor: ['tingkat_layanan', 'nama_tindakan', 'durasi_rawat']
  > Menguji Algoritma: LGBM...
    > Selesai. Waktu Latih: 2.53 detik. R2: 0.76354

--- Menguji Set Fitur: Model 2 (Top 5 Fitur) ---
    Kombinasi Prediktor: ['tingkat_layanan', 'nama_tindakan', 'durasi_rawat', 'prov_faskes', 'gender']
  > Menguji Algoritma: LGBM...
    > Selesai. Waktu Latih: 0.68 detik. R2: 0.78342

--- Menguji Set Fitur: Model 3 (Top 10 Fitur) ---
    Kombinasi Prediktor: ['tingkat_layanan', 'nama_tindakan', 'durasi_rawat', 'prov_faskes', 'gender', 'prov_peserta', 'tipe_faskes', 'bobot', 'nama_diagnosis', 'kab_kota_faskes']
  > Menguji Algoritma: LGBM...
    > Selesai. Waktu Latih: 3.92 detik. R2: 0.85074

--- Menguji Set Fitur: Model 4 (Top 15 Fitur) ---
    Kombinasi Prediktor: ['tingkat_layanan', 'nama_tindakan', 'durasi_rawat', 'prov_faskes', 'gender', 'prov_peserta', 'tipe_faskes', 

,Model,Kombinasi Prediktor,MAE,RMSE,R2
2,LGBM - Model 3 (Top 10 Fitur),"[tingkat_layanan, nama_tindakan, durasi_rawat,...",21351.975099,84269.140104,0.850737
4,LGBM - Model 5 (Semua 18 Fitur),"[hub_keluarga, status_pernikahan, segmen_peser...",21331.603637,89610.011203,0.831217
3,LGBM - Model 4 (Top 15 Fitur),"[tingkat_layanan, nama_tindakan, durasi_rawat,...",21445.958718,90422.137832,0.828144
1,LGBM - Model 2 (Top 5 Fitur),"[tingkat_layanan, nama_tindakan, durasi_rawat,...",27471.531944,101508.838145,0.783418
0,LGBM - Model 1 (Top 3 Fitur),"[tingkat_layanan, nama_tindakan, durasi_rawat]",28286.675733,106064.804168,0.763540


### **4.4 Pemilihan Model Terbaik, Pelatihan Model Final, dan Penyimpanan Model**

Model akan dipilih berdasarkan peringkat kinerja dan nilai metrik evaluasi utamanya. Namun, apabila nilai R² model mendekati sangat 1 (misalnya antara 0,98–1,0), kondisi tersebut justru dapat menjadi indikasi terjadinya overfitting atau bahkan data leakage, yaitu ketika model, secara sadar maupun tidak, memperoleh akses terhadap variabel prediktor yang terlalu informatif atau secara implisit merepresentasikan target (biaya_tagihan). Situasi ini membuat model tampak sangat akurat pada data pelatihan, tetapi gagal mempertahankan performanya saat diuji pada data baru (unseen data).

Oleh karena itu, model dengan performa terbaik tidak hanya dipilih berdasarkan skor tertinggi, tetapi juga melalui evaluasi stabilitas dan generalisasi. Setelah model terbaik terverifikasi mampu menjaga keseimbangan antara akurasi dan kemampuan generalisasi, model tersebut akan dilatih ulang menggunakan seluruh data pelatihan yang bersih dan teroptimasi untuk memastikan representasi yang paling komprehensif. Hasil akhir kemudian disimpan dalam format .pkl dengan nama final_best_model_LGBMRegressor.pkl untuk keperluan implementasi dan replikasi di tahap selanjutnya.

---


In [ ]:
import os

# ======================================================================
# --- Pilih Model Terbaik dari Hasil Perbandingan ---
# ======================================================================

df_results_sorted = df_lgbm_feature_comparison.reset_index(drop=True)

# Ambil informasi model terbaik
best_model_info = df_results_sorted.loc[0]
best_model_name = best_model_info["Model"]

best_model_features = best_model_info["Kombinasi Prediktor"]
best_model_r2_validation = best_model_info["R2"]

print("\n=== Model Terbaik Pilihan (dari Train/Test Split) ===")
print(f"Nama Model      : {best_model_name}")
print(f"Jumlah Fitur    : {len(best_model_features)}")
print(f"Estimasi R²     : {best_model_r2_validation:.4f}\n")
print(f"Fitur Digunakan : {best_model_features}\n")

# ======================================================================
# --- Latih Model Final (dengan Seluruh Data) ---
# ======================================================================

print(f"Memulai pelatihan model final ('{best_model_name}') pada SELURUH data...")

X_final = df_gabungan_reordered_clean[best_model_features]
y_final = df_gabungan_reordered_clean[target]

final_cat_features = [col for col in best_model_features if col in categorical_features]
print(f"Dari {len(best_model_features)} fitur, {len(final_cat_features)} terdeteksi sebagai kategorial.")

X_final_copy = X_final.copy()
for col in final_cat_features:
    X_final_copy[col] = X_final_copy[col].astype('category')

final_model_params = models_to_test['LGBM'].get_params()
final_model_params['verbosity'] = -1

final_model = lgbm.LGBMRegressor(**final_model_params)

print("Melatih model final LGBM...")
final_model.fit(
    X_final_copy,
    y_final,
    categorical_feature=final_cat_features,
    callbacks=[lgbm.log_evaluation(period=100)]
)
print("Pelatihan model final selesai.")

# ======================================================================
# --- Evaluasi Model Final dengan Cross-Validation ---
# ======================================================================

cv_scores = cross_val_score(final_model, X_final_copy, y_final, cv=5, scoring='r2')
print(f"R² rata-rata dari 5-fold CV : {np.mean(cv_scores):.4f}")
print(f"Standar deviasi R²          : {np.std(cv_scores):.4f}")

# ======================================================================
# --- Simpan Model Final ---
# ======================================================================

save_path = "/content/drive/MyDrive/Colab Notebooks/SESUAIKAN_DENGAN_PATH_DRIVE_ANDA.pkl"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

joblib.dump(final_model, save_path)
print(f"\n✅ Model final berhasil disimpan di:\n{save_path}")



=== Model Terbaik Pilihan (dari Train/Test Split) ===
Nama Model      : LGBM - Model 3 (Top 10 Fitur)
Jumlah Fitur    : 10
Estimasi R²     : 0.8507

Fitur Digunakan : ['tingkat_layanan', 'nama_tindakan', 'durasi_rawat', 'prov_faskes', 'gender', 'prov_peserta', 'tipe_faskes', 'bobot', 'nama_diagnosis', 'kab_kota_faskes']

Memulai pelatihan model final ('LGBM - Model 3 (Top 10 Fitur)') pada SELURUH data...
Dari 10 fitur, 8 terdeteksi sebagai kategorial.
Melatih model final LGBM...
Pelatihan model final selesai.
R² rata-rata dari 5-fold CV : 0.7655
Standar deviasi R²          : 0.0896

✅ Model final berhasil disimpan di:
/content/drive/MyDrive/Colab Notebooks/SESUAIKAN_DENGAN_PATH_DRIVE_ANDA.pkl


### **4.5 Melakukan Prediksi dengan Data Baru**
Dataset baru akan diprediksi menggunakan Model LGBM – Model 3 (Top 10 Fitur) yang dirancang khusus untuk memprediksi biaya_tagihan berdasarkan sepuluh fitur paling relevan. Karena model hanya mengenali dan memanfaatkan sepuluh fitur tersebut, maka meskipun dataset baru memiliki lebih banyak kolom, fitur-fitur tambahan yang tidak termasuk dalam sepuluh fitur utama tidak akan memengaruhi hasil prediksi,model secara otomatis akan mengabaikannya.

Hasil prediksi kemudian akan disajikan dalam dua bentuk tabel:

1. Hasil Prediksi (Semua Kolom Asli + Prediksi) – menampilkan keseluruhan kolom dari dataset awal  beserta kolom hasil prediksi.

2. Hasil Prediksi (Hanya 10 Fitur Model + Prediksi) – menampilkan hanya sepuluh fitur yang digunakan oleh model beserta kolom hasil prediksi untuk analisis yang lebih fokus dan ringkas.

---

In [ ]:
# ======================================================================
# --- Gunakan Model untuk Prediksi Data Baru ---
# ======================================================================

print("\n--- PREDIKSI MENGGUNAKAN DATA BARU DARI FILE ---")

new_data_path = "/content/drive/MyDrive/Colab Notebooks/SESUAIKAN_DENGAN_PATH_DRIVE_ANDA.csv"

try:
    # 1. Muat model
    loaded_model = joblib.load(save_path)
    print(f"Model '{save_path}' berhasil dimuat.")

    # 2. Muat data baru
    print(f"Mencoba memuat data baru dari: {new_data_path}")

    # Asumsi Anda sudah memperbaiki file CSV-nya (koma sebagai pemisah)
    new_data = pd.read_csv(new_data_path)

    print(f"Data baru berhasil dimuat. {len(new_data)} baris ditemukan.")

    # 3. Ambil fitur yang dibutuhkan dan TAMBAHKAN .copy()
    # Ini membuat DataFrame baru yang independen
    new_data_for_prediction = new_data[best_model_features].copy()

    # 3a. Pastikan data kategorial memiliki tipe yang benar
    for col in final_cat_features:
        if col in new_data_for_prediction.columns:
            new_data_for_prediction[col] = new_data_for_prediction[col].astype('category')

    # 4. Lakukan Prediksi
    # Model akan menerima data dengan tipe data (dtypes) yang
    # sama persis seperti saat latihan.
    predictions = loaded_model.predict(new_data_for_prediction)

    # 5. Tambahkan hasil prediksi ke DataFrame (data asli yang lengkap)
    new_data['prediksi_biaya_tagihan'] = np.round(predictions, 2)

    print("\n--- 1. Hasil Prediksi (Semua Kolom Asli + Prediksi) ---")
    display(new_data)

    print(f"\n--- 2. Hasil Prediksi (Hanya {len(best_model_features)} Fitur Model + Prediksi) ---")

    # Buat list kolom untuk tabel kedua
    kolom_prediktor_saja = best_model_features + ['prediksi_biaya_tagihan']

    # Tampilkan DataFrame HANYA dengan kolom-kolom tersebut
    display(new_data[kolom_prediktor_saja])

except FileNotFoundError:
    print(f"⚠️ Gagal: File data baru tidak ditemukan.")
    print(f"Pastikan Anda sudah meng-upload file ke path yang benar:")
    print(f"{new_data_path}")
except KeyError as e:
    print(f"⚠️ Gagal: Data baru Anda (dari file) kekurangan kolom yang dibutuhkan oleh model.")
    print(f"Kolom yang hilang: {e}")
    print(f"Pastikan file CSV Anda memiliki SEMUA {len(best_model_features)} kolom ini: {best_model_features}")
except Exception as e:
    print(f"Terjadi error yang tidak terduga: {e}")


--- PREDIKSI MENGGUNAKAN DATA BARU DARI FILE ---
Model '/content/drive/MyDrive/Colab Notebooks/SESUAIKAN_DENGAN_PATH_DRIVE_ANDA.pkl' berhasil dimuat.
Mencoba memuat data baru dari: /content/drive/MyDrive/Colab Notebooks/SESUAIKAN_DENGAN_PATH_DRIVE_ANDA.csv
Data baru berhasil dimuat. 20 baris ditemukan.

--- 1. Hasil Prediksi (Semua Kolom Asli + Prediksi) ---


,usia_pasien,hub_keluarga,gender,status_pernikahan,segmen_peserta,prov_peserta,kab_kota_peserta,bobot,durasi_rawat,kelas_rawat,prov_faskes,kab_kota_faskes,kepemilikan_faskes,jenis_faskes,tipe_faskes,tingkat_layanan,nama_diagnosis,nama_tindakan,prediksi_biaya_tagihan
0,78,Istri Peserta,Perempuan,Cerai,PBI APBD,Bali,Gianyar,1.20,3,Kelas II,Bali,Badung,TNI AD,Dokter umum,Laboratorium,RITP,Supervision of other normal pregnancy,Koreksi Paket Persalinan per Vaginam normal (o...,105739.41
1,33,Keluarga Tambahan Peserta,Laki-laki,Kawin,PBI APBN,Jawa Barat,Depok,1.79,8,Missing,Jawa Barat,Cirebon,TNI AL,Puskesmas,Rawat Inap,RITP,Typhoid fever,Pelayanan PNC 1 (Satu),196536.73
2,2,Peserta,Perempuan,Tidak terdefinisi,Missing,DKI Jakarta,Jakarta Utara,1.68,7,Kelas II,Jawa Timur,Madiun,Pemerintah provinsi,Missing,Rawat Inap,Promotif,Contraceptive management,Kolesterol Trigliserida,63192.78
3,41,Istri Peserta,Perempuan,Tidak terdefinisi,Bukan pekerja,DKI Jakarta,Jakarta Pusat,1.41,2,Kelas III,Bali,Tabanan,Swasta,Dokter umum,Faskes IVA/Pap Smear,RJTP,"Single spontaneous delivery, unspecified",Evakuasi medis / Ambulans Air,1944066.86
4,17,Suami Peserta,Laki-laki,Belum kawin,Missing,Jawa Timur,Surabaya,1.32,4,Kelas I,Jawa Timur,Madiun,BUMN,Puskesmas,RS Kelas D Pratama,RITP,Dyspnoea,Paket persalinan per vaginam normal,602083.17
5,37,Suami Peserta,Laki-laki,Kawin,PBI APBD,Jawa Timur,Madiun,1.70,12,Kelas II,Jawa Tengah,Semarang,BUMN,Puskesmas,RS Kelas D Pratama,RITP,Contraceptive management,Koreksi Paket Persalinan per Vaginam normal (o...,77566.72
6,24,Anak Peserta,Laki-laki,Tidak terdefinisi,Bukan pekerja,Bali,Badung,1.27,10,Kelas III,DKI Jakarta,Jakarta Utara,TNI AL,Missing,Klinik Rawat Inap,RJTP,"Fever, unspecified",Evakuasi medis / Ambulans Darat,799401.77
7,25,Keluarga Tambahan Peserta,Perempuan,Cerai,PBI APBD,Jawa Barat,Bekasi,0.53,6,Kelas II,Jawa Timur,Malang,Missing,Dokter umum,Faskes IVA/Pap Smear,RITP,Dyspepsia,Evakuasi medis / Ambulans Air,795195.81
8,18,Anak Peserta,Laki-laki,Tidak terdefinisi,PBI APBD,DKI Jakarta,Jakarta Timur,1.83,3,Kelas II,Bali,Badung,TNI AU,Dokter umum,Faskes IVA/Pap Smear,Promotif,Supervision of other normal pregnancy,Pelayanan ANC 4 (Empat),50754.53
9,67,Suami Peserta,Laki-laki,Belum kawin,PBPU,DKI Jakarta,Jakarta Utara,1.05,14,Missing,Bali,Bangli,Missing,Klinik pratama,Rawat Inap,Promotif,"Secondary hypertension, unspecified",Rawat Inap di R. Perawatan Biasa,782822.35



--- 2. Hasil Prediksi (Hanya 10 Fitur Model + Prediksi) ---


,tingkat_layanan,nama_tindakan,durasi_rawat,prov_faskes,gender,prov_peserta,tipe_faskes,bobot,nama_diagnosis,kab_kota_faskes,prediksi_biaya_tagihan
0,RITP,Koreksi Paket Persalinan per Vaginam normal (o...,3,Bali,Perempuan,Bali,Laboratorium,1.20,Supervision of other normal pregnancy,Badung,105739.41
1,RITP,Pelayanan PNC 1 (Satu),8,Jawa Barat,Laki-laki,Jawa Barat,Rawat Inap,1.79,Typhoid fever,Cirebon,196536.73
2,Promotif,Kolesterol Trigliserida,7,Jawa Timur,Perempuan,DKI Jakarta,Rawat Inap,1.68,Contraceptive management,Madiun,63192.78
3,RJTP,Evakuasi medis / Ambulans Air,2,Bali,Perempuan,DKI Jakarta,Faskes IVA/Pap Smear,1.41,"Single spontaneous delivery, unspecified",Tabanan,1944066.86
4,RITP,Paket persalinan per vaginam normal,4,Jawa Timur,Laki-laki,Jawa Timur,RS Kelas D Pratama,1.32,Dyspnoea,Madiun,602083.17
5,RITP,Koreksi Paket Persalinan per Vaginam normal (o...,12,Jawa Tengah,Laki-laki,Jawa Timur,RS Kelas D Pratama,1.70,Contraceptive management,Semarang,77566.72
6,RJTP,Evakuasi medis / Ambulans Darat,10,DKI Jakarta,Laki-laki,Bali,Klinik Rawat Inap,1.27,"Fever, unspecified",Jakarta Utara,799401.77
7,RITP,Evakuasi medis / Ambulans Air,6,Jawa Timur,Perempuan,Jawa Barat,Faskes IVA/Pap Smear,0.53,Dyspepsia,Malang,795195.81
8,Promotif,Pelayanan ANC 4 (Empat),3,Bali,Laki-laki,DKI Jakarta,Faskes IVA/Pap Smear,1.83,Supervision of other normal pregnancy,Badung,50754.53
9,Promotif,Rawat Inap di R. Perawatan Biasa,14,Bali,Laki-laki,DKI Jakarta,Rawat Inap,1.05,"Secondary hypertension, unspecified",Bangli,782822.35
